<div class = "row">
    <div class = "colums">
        <img src="..\trecslogo.png" align="left" alt="Drawing" width ="60"/>    
    </div>
    <div class = "colums">
        <img src="..\asrlogo.png" align="right" alt="Drawing" width ="175"/>
    </div>    
</div>

# RISICOMARGE LEVEN

## Algemeen

### Import en instellingen

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from bokeh.plotting import figure, output_file, ColumnDataSource, output_notebook
from bokeh.models import HoverTool, NumeralTickFormatter, FactorRange
from bokeh.io import show
output_notebook(hide_banner=True)
import timeit
import math

Links uitlijnen tabellen

In [2]:
%%html
<style>
    table {
        display: inline-block
    }
</style>

Bij printen van een dataframe wordt slechts een beperkt aantal rijen getoond.

In [3]:
pd.set_option('display.max_rows', 300)

## Initialisatie parameters

In [4]:
# data
datumKwartaal = 20210630
datumActueel = 20210630 # in dit notebook veronderstel ik dat de actuele datum dezelfde is als de kwartaal datum
data = (datumKwartaal, datumActueel)

# duratie bonds en percentage bonds in fonds tbv geschokte waarde UL fonds op t=0
durationBonds = 5
percentageBondsIndividueel = 0.346506
percentageBondsCollectief = 0.360153

# valutakoersen
EURAUD = 1.57965
EURUSD = 1.1859
EURZAR = 16.93315

# parameters tbv operationeel risico
percentageOperationeelRisicoPremie = 0.04
percentageOperationeelRisicoVoorziening = 0.0045
percentageOperationeelRisicoUnitLinkedKosten = 0.25
ORKostenULLeven = 38168411.26
ORKostenULPensioenen = 26129663.25

# iboxx spread tbv meerwaarde spaarlossen
iboxxSpread = 0.000417757957262977

# parameters tbv risk drivers
schokKosten = 0.1 # PC_Schok_kosten_SII
schokInflatie = 0.01 # PC_Schok_inflatie_SII
ORPremieRiskDriver = 0.03 # PC_OR_Nom_Premie_RiskDriver
ORLiabilityValueRiskDriver = 0.003 # PC_OR_Nom_BEL_RiskDriver

# parameter tbv kostenrisico
beleggingskostenBijtellingPercentage = 0.1

# naam van de basis curve
renteCurveBasis = 'SII_basis'

# parameters tbv risicomarge
CoC = 0.06
correlatiesSCRLeven = np.array([
                            [1, -0.25, 0.25, 0, 0.25, 0, 0.25],
                            [-0.25, 1, 0, 0.25, 0.25, 0.25, 0],
                            [0.25, 0, 1, 0, 0.5, 0, 0.25],
                            [0, 0.25, 0, 1, 0.5, 0, 0.25],
                            [0.25, 0.25, 0.5, 0.5, 1, 0.5, 0.25],
                            [0, 0.25, 0, 0, 0.5, 1, 0],
                            [0.25, 0, 0.25, 0.25, 0.25, 0, 1]
                        ])

## Importeer en bewerk data

### Configuraties
Deze data dient in principe eenmalig geconfigureerd te worden. Niet ieder kwartaal zullen deze gegevens worden aangepast.

#### Risico types
De volgende verzekeringstechnische risico's worden onderscheiden. Zie hoofdstuk Documentatie voor verdere toelichting. 

In [5]:
risicoTypes = ('201', '202', '203', '204', '205', '206', '207', '208', '209', '210')

#### Modelpunten

In [6]:
col_list = ['modelpunt']
df_modelpuntNamen = pd.read_excel(r"databestanden SSC/Modelpunten/configuratiesModelpunten.xlsx", usecols = col_list)
modelpuntNamen = tuple(df_modelpuntNamen['modelpunt'])
modelpuntNamen
# df_modelpuntNamen.to_excel('databestanden SSC\TRECS uitvraag\modelpuntNamen.xlsx')

('ANWH',
 'EABA',
 'EABB',
 'EABC',
 'EABD',
 'EABE',
 'EABF',
 'EABG',
 'EABJ',
 'EABK',
 'EABL',
 'EABM',
 'EABT',
 'EABV',
 'EACG',
 'EACN',
 'EADP',
 'EADV',
 'EANW',
 'EAOA',
 'EAOB',
 'EAOC',
 'EAOD',
 'EAOE',
 'EAOF',
 'EAOG',
 'EAOJ',
 'EAOK',
 'EAOL',
 'EAOM',
 'EAOT',
 'EAOV',
 'EAPF',
 'EAPG',
 'EARA',
 'EARB',
 'EARC',
 'EARD',
 'EARE',
 'EARF',
 'EARG',
 'EARJ',
 'EARK',
 'EARL',
 'EARM',
 'EART',
 'EARV',
 'EASD',
 'EASV',
 'EAZF',
 'EAZH',
 'EAZN',
 'EAZO',
 'EAZT',
 'EONW',
 'EUNW',
 'EUWD',
 'EUWV',
 'FRGG',
 'FRGL',
 'FRGM',
 'FRGR',
 'FRKL',
 'FRKR',
 'FRNM',
 'GAGS',
 'GASA',
 'IAAN',
 'IAAW',
 'IABE',
 'IACD',
 'IACV',
 'IADS',
 'IAFN',
 'IAFW',
 'IAGA',
 'IAIA',
 'IAIN',
 'IAKN',
 'IAKW',
 'IAMA',
 'IAMI',
 'IANA',
 'IANN',
 'IANW',
 'IAPA',
 'IAPN',
 'IARN',
 'IARW',
 'IATD',
 'IAUA',
 'IAUN',
 'IAUW',
 'IAVA',
 'IAVD',
 'IAVO',
 'IAVV',
 'IAXA',
 'IAXB',
 'IAXC',
 'IAXD',
 'IAXE',
 'IAXF',
 'IAXN',
 'IAXP',
 'IAXQ',
 'IAZA',
 'IAZI',
 'IBNM',
 'IBNW',
 'IBSH',
 

#### Valuta

In [7]:
col_list = ['modelpunt', 'currency']
df_valuta = pd.read_excel(r"databestanden SSC/Modelpunten/configuratiesModelpunten.xlsx", usecols = col_list)
df_valuta

,modelpunt,currency
0,ANWH,EUR
1,EABA,EUR
2,EABB,EUR
3,EABC,EUR
4,EABD,EUR
5,EABE,EUR
6,EABF,EUR
7,EABG,EUR
8,EABJ,EUR
9,EABK,EUR


#### Methoden risk drivers
Voor het berekenen van de risicomarge is een projectie van de verschillende verzekeringstechnische risicokapitalen nodig. SSC doet dit pragmatisch met risk drivers. Deze drivers zijn gebaseerd op actuariële grootheden als ontwikkeling van de boekhoudkundige voorzieningen en uitkeringen.

In [8]:
col_list = ['modelpunt', 'Methode_RiskDriver_Kortleven', 'Methode_RiskDriver_Langleven', 'Methode_RiskDriver_AO', 'Methode_RiskDriver_LapseDown', 'Methode_RiskDriver_LapseUp', 'Methode_RiskDriver_LapseMass', 'Ind_VVP_Riskdriver_Corrigeren']
df_methodenRiskDrivers = pd.read_excel(r"databestanden SSC/Modelpunten/configuratiesModelpunten.xlsx", usecols = col_list)
df_methodenRiskDrivers.rename(columns={'Methode_RiskDriver_Kortleven': '202', 'Methode_RiskDriver_Langleven': '203', 'Methode_RiskDriver_AO': '204', 'Methode_RiskDriver_LapseDown': '205', 'Methode_RiskDriver_LapseUp': '206', 'Methode_RiskDriver_LapseMass': '207', 'Ind_VVP_Riskdriver_Corrigeren': 'indicatorCorrectie'}, inplace=True)
df_methodenRiskDrivers

,modelpunt,202,203,204,205,206,207,indicatorCorrectie
0,ANWH,FN_RISPREM,VVP (Math. Reserve),FN_PREM_IV,Afkoop Uitkeringen,Afkoop Uitkeringen,Afkoop Uitkeringen,0
1,EABA,FN_RISPREM,VVP (Math. Reserve),FN_PREM_IV,Afkoop Uitkeringen,Afkoop Uitkeringen,CW(KD+PR),0
2,EABB,FN_RISPREM,VVP (Math. Reserve),FN_PREM_IV,Afkoop Uitkeringen,Afkoop Uitkeringen,CW(KD+PR),0
3,EABC,FN_RISPREM,VVP (Math. Reserve),FN_PREM_IV,Afkoop Uitkeringen,Afkoop Uitkeringen,CW(KD+PR),0
4,EABD,FN_RISPREM,VVP (Math. Reserve),FN_PREM_IV,Afkoop Uitkeringen,Afkoop Uitkeringen,CW(KD+PR),0
5,EABE,FN_RISPREM,VVP (Math. Reserve),FN_PREM_IV,Afkoop Uitkeringen,Afkoop Uitkeringen,CW(KD+PR),0
6,EABF,FN_RISPREM,VVP (Math. Reserve),FN_PREM_IV,Afkoop Uitkeringen,Afkoop Uitkeringen,CW(KD+PR),0
7,EABG,FN_RISPREM,VVP (Math. Reserve),FN_PREM_IV,Afkoop Uitkeringen,Afkoop Uitkeringen,CW(KD+PR),0
8,EABJ,FN_RISPREM,VVP (Math. Reserve),FN_PREM_IV,Afkoop Uitkeringen,Afkoop Uitkeringen,CW(KD+PR),0
9,EABK,FN_RISPREM,VVP (Math. Reserve),FN_PREM_IV,Afkoop Uitkeringen,Afkoop Uitkeringen,CW(KD+PR),0


#### Product soorten
De berekening van de verzekeringstechnische risico's hangt af van het soort product dat wordt vertegenwoordigd door het modelpunt. Onderscheid wordt gemaakt naar productlijn, QRT niveau, unit linked, nominaal, collectief, individueel en uitvaart. Voor operationeel risico wordt rekening gehouden met modelpunten die zullen 'muteren' van unit linked naar nominaal. Navragen bij SSC wat dit laatste exact betekent.

In [9]:
col_list = ['modelpunt', 'productlijn', 'winstdelend', 'QRT_niveau', 'Ind_OR_UL_naar_Nom', 'ULGarantie']
df_productSoorten = pd.read_excel(r"databestanden SSC/Modelpunten/configuratiesModelpunten.xlsx", usecols = col_list)

df_productSoorten.loc[df_productSoorten['winstdelend'] == 2, 'unitLinked'] = 1
df_productSoorten.loc[(df_productSoorten['unitLinked'] == 1) & (df_productSoorten['productlijn'] == 3), 'unitLinkedCollectief'] = 1
df_productSoorten.loc[df_productSoorten['winstdelend'] != 2, 'nominaal'] = 1
df_productSoorten.loc[(df_productSoorten['nominaal'] == 1) & (df_productSoorten['productlijn'] == 3), 'nominaalCollectief'] = 1
df_productSoorten.loc[(df_productSoorten['nominaal'] == 1) & (df_productSoorten['productlijn'] == 1), 'nominaalUitvaart'] = 1
df_productSoorten.loc[(df_productSoorten['nominaal'] == 1) & (df_productSoorten['nominaalCollectief'] != 1) & (df_productSoorten['nominaalUitvaart'] != 1), 'nominaalRest'] = 1

# indeling tbv operationeel risico
df_productSoorten.loc[df_productSoorten['Ind_OR_UL_naar_Nom'] == 1, 'unitLinkedNaarNominaalOperationeelRisico'] = 1
df_productSoorten.loc[(df_productSoorten['nominaal'] == 1) | (df_productSoorten['unitLinkedNaarNominaalOperationeelRisico'] == 1), 'nominaalOperationeelRisico'] = 1
df_productSoorten.loc[df_productSoorten['nominaalOperationeelRisico'] != 1, 'unitLinkedOperationeelRisico'] = 1
df_productSoorten.loc[(df_productSoorten['nominaalOperationeelRisico'] == 1) & (df_productSoorten['productlijn'] == 2), 'nominaalLevenOperationeelRisico'] = 1
df_productSoorten.loc[(df_productSoorten['nominaalOperationeelRisico'] == 1) & (df_productSoorten['productlijn'] == 3), 'nominaalCollectiefOperationeelRisico'] = 1

df_productSoorten['weight'] = np.where(df_productSoorten['productlijn'] == 3, 0.7, 0.4)

df_productSoorten = df_productSoorten.drop(columns = ['winstdelend', 'Ind_OR_UL_naar_Nom'])

pd.options.display.float_format = '{:,.1f}'.format
df_productSoorten
#df_productSoorten.to_excel('output2.xlsx')

,modelpunt,productlijn,QRT_niveau,ULGarantie,unitLinked,unitLinkedCollectief,nominaal,nominaalCollectief,nominaalUitvaart,nominaalRest,unitLinkedNaarNominaalOperationeelRisico,nominaalOperationeelRisico,unitLinkedOperationeelRisico,nominaalLevenOperationeelRisico,nominaalCollectiefOperationeelRisico,weight
0,ANWH,3,3,0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,0.7
1,EABA,3,1,0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,0.7
2,EABB,3,1,0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,0.7
3,EABC,3,1,0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,0.7
4,EABD,3,1,0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,0.7
5,EABE,3,1,0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,0.7
6,EABF,3,1,0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,0.7
7,EABG,3,1,0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,0.7
8,EABJ,3,1,0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,0.7
9,EABK,3,1,0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,0.7


#### Indicatoren kasstromen

In [10]:
df_kasstromenIndicatoren = pd.read_excel(r"databestanden SSC/Kasstromen/Verplichtingenteken.xlsx")
df_kasstromenIndicatoren

,VAR_NAME,CF,BV,CFU,CFC,CFP
0,MATH_RES_IF,0,1,0,0,0
1,INV_EXP,-1,0,0,-1,0
2,ANNUAL_PREM,1,0,0,0,1
3,SINGLE_PREM,1,0,0,0,1
4,NET_PREMIUM,0,0,0,0,0
5,SUM_ASSURED,0,0,0,0,0
6,PRICE_MORTALITY,0,0,0,0,0
7,EXP_MORTALITY,0,0,0,0,0
8,INIT_EXP,-1,0,0,-1,0
9,REN_EXP,-1,0,0,-1,0


### Posities modelpunten
Met posities modelpunten wordt de data bedoeld die periodiek (per modelpunt) zal worden aangepast, waarbij de kasstromen buiten scope zijn.

#### Beleggingskosten percentage

In [11]:
col_list = ['datum', 'modelpunt', 'inv_exp_pc']
df_beleggingskostenPercentages = pd.read_excel(r"databestanden SSC/Modelpunten/positiesModelpunten.xlsx", usecols = col_list)
df_beleggingskostenPercentages.rename(columns={'inv_exp_pc': 'beleggingskostenPercentage'}, inplace=True)

pd.options.display.float_format = '{:,.4f}'.format
df_beleggingskostenPercentages

,datum,modelpunt,beleggingskostenPercentage
0,20210630,ANWH,0.0550
1,20210630,EABA,0.0550
2,20210630,EABB,0.0550
3,20210630,EABC,0.0550
4,20210630,EABD,0.0550
5,20210630,EABE,0.0550
6,20210630,EABF,0.0550
7,20210630,EABG,0.0550
8,20210630,EABJ,0.0550
9,20210630,EABK,0.0550


#### Waarden voorzieningen per VTR
OSM / Gamma heeft per modelpunt de waarde van de best estimate voorziening uitgerekend, niet alleen in het basis scenario maar ook voor alle verzekeringstechnische risico's. De waarden die door het notebook worden uitgerekend zullen afwijken van de OSM / Gamma waarden, o.a. als gevolg van het feit dat winstdelingsopties niet worden meegenomen dan wel worden benaderd met een replicatieformule. De berekende waarden door het notebook worden in het economische basis scenario geijkt met de waarden die door OSM / Gamma zijn berekend. Alle waarden betreffen dus het economische basis scenario.

In [12]:
col_list = ['datum', 'modelpunt', 'BEL_OSM_Basis', 'BEL_Hand', 'BEL_OSM_Kortleven', 'BEL_OSM_Langleven', 'BEL_OSM_AO', 'BEL_OSM_LapseDown', 'BEL_OSM_LapseUp', 'BEL_OSM_LapseMass', 'BEL_OSM_Kosten', 'BEL_OSM_Cat', 'VVP_LapseMass_Hand_Corr']
df_waardenVoorzieningenGamma = pd.read_excel(r"databestanden SSC/Modelpunten/positiesModelpunten.xlsx", usecols = col_list)
df_waardenVoorzieningenGamma.rename(columns={'BEL_OSM_Basis': '201', 'BEL_Hand': '201Hand', 'BEL_OSM_Kortleven': '202', 'BEL_OSM_Langleven': '203', 'BEL_OSM_AO': '204', 'BEL_OSM_LapseDown': '205', 'BEL_OSM_LapseUp': '206', 'BEL_OSM_LapseMass': '207', 'BEL_OSM_Kosten': '208', 'BEL_OSM_Cat': '209', 'VVP_LapseMass_Hand_Corr': '207Hand'}, inplace=True)

df_waardenVoorzieningenGamma

,datum,modelpunt,201Hand,201,202,203,204,205,206,207,208,209,207Hand
0,20210630,ANWH,0.0000,"542,251.5375","596,639.5665","468,773.3986","542,251.5375","542,251.5375","542,251.5375","542,251.5375","689,837.7505","10,054,864.0894",0.0000
1,20210630,EABA,-0.0040,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0008,0.0007,0.0000
2,20210630,EABB,-0.0040,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0008,0.0007,0.0000
3,20210630,EABC,-0.0040,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0008,0.0007,0.0000
4,20210630,EABD,"-289,796.9350","120,458,810.5513","116,562,254.5487","126,583,481.0035","120,458,810.5513","120,458,810.5513","120,458,810.5513","120,458,810.5513","122,986,888.4386","120,372,348.8912","-1,240,681.1676"
5,20210630,EABE,-0.0040,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0008,0.0007,0.0000
6,20210630,EABF,-0.0040,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0008,0.0007,0.0000
7,20210630,EABG,-0.0040,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0008,0.0007,0.0000
8,20210630,EABJ,-0.0040,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0008,0.0007,0.0000
9,20210630,EABK,-0.0040,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0008,0.0007,0.0000


#### Meerwaarden spaarlossen per VTR
OSM / Gamma heeft per modelpunt de meerwaarde van de spaarlossen uitgerekend (indien aanwezig), niet alleen in het basis scenario maar ook voor alle verzekeringstechnische risico's. De waarden die door het notebook worden uitgerekend zullen afwijken van de OSM / Gamma waarden. De berekende waarden door het notebook worden in het economische basis scenario geijkt met de waarden die door OSM / Gamma zijn berekend. Alle waarden betreffen dus het economische basis scenario.

In [13]:
col_list = ['datum', 'modelpunt', 'MWSL_Basis', 'MWSL_Kortleven', 'MWSL_Langleven', 'MWSL_AO', 'MWSL_LapseDown', 'MWSL_LapseUp', 'MWSL_LapseMass', 'MWSL_Kosten', 'MWSL_Cat']
df_meerwaardeSpaarlossenGamma = pd.read_excel(r"databestanden SSC/Modelpunten/positiesModelpunten.xlsx", usecols = col_list)
df_meerwaardeSpaarlossenGamma.rename(columns={'MWSL_Basis': '201', 'MWSL_Kortleven': '202', 'MWSL_Langleven': '203', 'MWSL_AO': '204', 'MWSL_LapseDown': '205', 'MWSL_LapseUp': '206', 'MWSL_LapseMass': '207', 'MWSL_Kosten': '208', 'MWSL_Cat': '209'}, inplace=True)

df_meerwaardeSpaarlossenGamma

,datum,modelpunt,201,202,203,204,205,206,207,208,209
0,20210630,ANWH,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,20210630,EABA,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,20210630,EABB,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,20210630,EABC,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,20210630,EABD,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
5,20210630,EABE,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
6,20210630,EABF,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
7,20210630,EABG,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
8,20210630,EABJ,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
9,20210630,EABK,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


#### Afkoop- en premie vrijmaak vloeren

In [14]:
col_list = ['datum', 'modelpunt', 'AfkW_Tool', 'Meth_Afk', 'Prem_Vrij_Vloer']
df_vloerenGamma = pd.read_excel(r"databestanden SSC/Modelpunten/positiesModelpunten.xlsx", usecols = col_list)
df_vloerenGamma.rename(columns={'AfkW_Tool': 'afkoopVloer', 'Meth_Afk': 'afkoopMethode', 'Prem_Vrij_Vloer': 'premieVrijmaakVloer'}, inplace=True)
df_vloerenGamma['afkoopIndicator'] = np.where(df_vloerenGamma['afkoopMethode'] != 3, 1, 0)
df_vloerenGamma

,datum,modelpunt,afkoopVloer,afkoopMethode,premieVrijmaakVloer,afkoopIndicator
0,20210630,ANWH,0.0000,1,0.0000,1
1,20210630,EABA,0.0000,1,0.0000,1
2,20210630,EABB,0.0000,1,0.0000,1
3,20210630,EABC,0.0000,1,0.0000,1
4,20210630,EABD,0.0000,1,0.0000,1
5,20210630,EABE,0.0000,1,0.0000,1
6,20210630,EABF,0.0000,1,0.0000,1
7,20210630,EABG,0.0000,1,0.0000,1
8,20210630,EABJ,0.0000,1,0.0000,1
9,20210630,EABK,0.0000,1,0.0000,1


### Kasstromen modelpunten

#### Import data kasstromen

In [15]:
xls = pd.ExcelFile(r"databestanden SSC/Kasstromen/Geschokte kasstromen 2021Q2_FRMMB.xlsx")
df_RUN_201_DET_CF_L = pd.read_excel(xls, 'RUN_201_DET_CF_L', decimal = '.')
df_RUN_201_DET_CF_L['risicoType'] = '201'
df_RUN_202_DET_CF_L = pd.read_excel(xls, 'RUN_202_DET_CF_L', decimal = '.')
df_RUN_202_DET_CF_L['risicoType'] = '202'
df_RUN_203_DET_CF_L = pd.read_excel(xls, 'RUN_203_DET_CF_L', decimal = '.')
df_RUN_203_DET_CF_L['risicoType'] = '203'
df_RUN_204_DET_CF_L = pd.read_excel(xls, 'RUN_204_DET_CF_L', decimal = '.')
df_RUN_204_DET_CF_L['risicoType'] = '204'
df_RUN_205_DET_CF_L = pd.read_excel(xls, 'RUN_205_DET_CF_L', decimal = '.')
df_RUN_205_DET_CF_L['risicoType'] = '205'
df_RUN_206_DET_CF_L = pd.read_excel(xls, 'RUN_206_DET_CF_L', decimal = '.')
df_RUN_206_DET_CF_L['risicoType'] = '206'
df_RUN_207_DET_CF_L = pd.read_excel(xls, 'RUN_207_DET_CF_L', decimal = '.')
df_RUN_207_DET_CF_L['risicoType'] = '207'
df_RUN_208_DET_CF_L = pd.read_excel(xls, 'RUN_208_DET_CF_L', decimal = '.')
df_RUN_208_DET_CF_L['risicoType'] = '208'
df_RUN_209_DET_CF_L = pd.read_excel(xls, 'RUN_209_DET_CF_L', decimal = '.')
df_RUN_209_DET_CF_L['risicoType'] = '209'
df_RUN_201_DET_CF_P = pd.read_excel(xls, 'RUN_201_DET_CF_P', decimal = '.')
df_RUN_201_DET_CF_P['risicoType'] = '201'
df_RUN_202_DET_CF_P = pd.read_excel(xls, 'RUN_202_DET_CF_P', decimal = '.')
df_RUN_202_DET_CF_P['risicoType'] = '202'
df_RUN_203_DET_CF_P = pd.read_excel(xls, 'RUN_203_DET_CF_P', decimal = '.')
df_RUN_203_DET_CF_P['risicoType'] = '203'
df_RUN_204_DET_CF_P = pd.read_excel(xls, 'RUN_204_DET_CF_P', decimal = '.')
df_RUN_204_DET_CF_P['risicoType'] = '204'
df_RUN_205_DET_CF_P = pd.read_excel(xls, 'RUN_205_DET_CF_P', decimal = '.')
df_RUN_205_DET_CF_P['risicoType'] = '205'
df_RUN_206_DET_CF_P = pd.read_excel(xls, 'RUN_206_DET_CF_P', decimal = '.')
df_RUN_206_DET_CF_P['risicoType'] = '206'
df_RUN_207_DET_CF_P = pd.read_excel(xls, 'RUN_207_DET_CF_P', decimal = '.')
df_RUN_207_DET_CF_P['risicoType'] = '207'
df_RUN_208_DET_CF_P = pd.read_excel(xls, 'RUN_208_DET_CF_P', decimal = '.')
df_RUN_208_DET_CF_P['risicoType'] = '208'
df_RUN_209_DET_CF_P = pd.read_excel(xls, 'RUN_209_DET_CF_P', decimal = '.')
df_RUN_209_DET_CF_P['risicoType'] = '209'
dfList = [df_RUN_201_DET_CF_L,df_RUN_202_DET_CF_L,df_RUN_203_DET_CF_L,df_RUN_204_DET_CF_L,df_RUN_205_DET_CF_L,df_RUN_206_DET_CF_L,df_RUN_207_DET_CF_L,df_RUN_208_DET_CF_L,df_RUN_209_DET_CF_L,df_RUN_201_DET_CF_P,df_RUN_202_DET_CF_P,df_RUN_203_DET_CF_P,df_RUN_204_DET_CF_P,df_RUN_205_DET_CF_P,df_RUN_206_DET_CF_P,df_RUN_207_DET_CF_P,df_RUN_208_DET_CF_P,df_RUN_209_DET_CF_P]
df_kasstromen = pd.concat(dfList)
df_kasstromen = df_kasstromen.reset_index(drop=True)
df_kasstromen['datum'] = datumKwartaal
datum_col = df_kasstromen.pop('datum')
df_kasstromen.insert(0, 'datum', datum_col)
risico_col = df_kasstromen.pop('risicoType')
df_kasstromen.insert(1, 'risicoType', risico_col)
df_kasstromen = pd.merge(df_kasstromen, df_kasstromenIndicatoren, how = 'left', on = ['VAR_NAME'])
df_kasstromen.rename(columns={' LIAB_NAME': 'modelpunt'}, inplace=True)
df_kasstromen
#df_kasstromen.to_excel('output.xlsx')

,datum,risicoType,modelpunt,VAR_NAME,0,12,24,36,48,60,...,1140,1152,1164,1176,1188,CF,BV,CFU,CFC,CFP
0,20210630,201,IAAN,MATH_RES_IF,"24,763,560.7800","24,830,586.1000","24,781,392.3700","24,676,687.7300","24,519,225.7900","24,310,764.1000",...,24.7900,13.0600,7.3100,4.0100,1.9700,0,1,0,0,0
1,20210630,201,IAAN,INV_EXP,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,-1,0,0,-1,0
2,20210630,201,IAAN,ANNUAL_PREM,0.0000,"344,943.0200","309,316.6500","277,577.2100","248,671.0500","225,476.2900",...,0.0000,0.0000,0.0000,0.0000,0.0000,1,0,0,0,1
3,20210630,201,IAAN,SINGLE_PREM,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,1,0,0,0,1
4,20210630,201,IAAN,NET_PREMIUM,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84937,20210630,209,EAPF,RD_NO_POLS_PV,0.0000,"1,470.0300","1,461.2700","1,449.3200","1,436.4300","1,426.3900",...,0.0000,0.0000,0.0000,0.0000,0.0000,0,0,0,0,0
84938,20210630,209,EAPF,RD_SURR_OUTGO,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0,0,0,0,0
84939,20210630,209,EAPF,D_OPT_VALUE_BS,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0,0,0,0,0
84940,20210630,209,EAPF,OPT_OUTGO_INT,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,-1,0,0,-1,0


#### Projectie boekwaarde voorziening

In [16]:
df_projectieBoekwaardeVoorziening = df_kasstromen.copy()
df_projectieBoekwaardeVoorziening.iloc[:, 4:104] = df_projectieBoekwaardeVoorziening.iloc[:, 4:104].multiply(df_projectieBoekwaardeVoorziening['BV'], axis = "index")
df_projectieBoekwaardeVoorziening = df_projectieBoekwaardeVoorziening.groupby(['datum', 'risicoType', 'modelpunt']).sum().reset_index()
df_projectieBoekwaardeVoorziening = df_projectieBoekwaardeVoorziening.drop(columns = ['CF', 'BV', 'CFU', 'CFC', 'CFP'])
df_projectieBoekwaardeVoorziening = df_projectieBoekwaardeVoorziening.reset_index(drop=True)
df_projectieBoekwaardeVoorziening

,datum,risicoType,modelpunt,0,12,24,36,48,60,72,...,1080,1092,1104,1116,1128,1140,1152,1164,1176,1188
0,20210630,201,ANWH,"1,621,363.0200","1,351,135.8500","1,080,908.6800","810,681.5100","540,454.3400","482,864.6400","430,281.7000",...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,20210630,201,EABA,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,20210630,201,EABB,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,20210630,201,EABC,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,20210630,201,EABD,"72,661,409.3200","73,033,772.3500","73,290,851.3400","73,416,653.4000","73,453,075.9300","73,280,567.1700","73,029,890.7300",...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2173,20210630,209,ZOVZ,"73,207,284.5500","73,902,363.4400","74,412,819.1100","74,741,805.9700","74,927,574.6400","74,999,602.7200","74,929,895.3700",...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2174,20210630,209,ZPFA,"2,359,306,994.0700","2,329,831,938.7800","2,294,680,033.2600","2,256,613,687.2300","2,215,568,201.4100","2,171,244,386.1400","2,123,844,589.0200",...,0.0600,0.0200,0.0100,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2175,20210630,209,ZPKS,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2176,20210630,209,ZPPK,"15,435,778.2500","17,497,593.6700","19,576,647.5400","19,848,870.9300","20,212,912.4600","20,564,472.3100","20,907,114.8600",...,2.8900,1.2300,0.5100,0.2000,0.0800,0.0300,0.0100,0.0000,0.0000,0.0000


#### Kasstromen totaal

In [17]:
df_kasstromenTotaal = df_kasstromen.copy()
df_kasstromenTotaal.iloc[:, 4:104] = df_kasstromenTotaal.iloc[:, 4:104].multiply(df_kasstromenTotaal['CF'], axis = "index")
df_kasstromenTotaal = df_kasstromenTotaal.groupby(['datum', 'risicoType', 'modelpunt']).sum().reset_index()
df_kasstromenTotaal = df_kasstromenTotaal.drop(columns = ['CF', 'BV', 'CFU', 'CFC', 'CFP'])
df_kasstromenTotaal = df_kasstromenTotaal.reset_index(drop=True)
df_kasstromenTotaal

,datum,risicoType,modelpunt,0,12,24,36,48,60,72,...,1080,1092,1104,1116,1128,1140,1152,1164,1176,1188
0,20210630,201,ANWH,0.0000,"8,877,271.5600","-502,767.9200","-823,523.8200","-794,530.0600","-746,552.8500","-706,750.8000",...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,20210630,201,EABA,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,20210630,201,EABB,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,20210630,201,EABC,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,20210630,201,EABD,0.0000,"-1,861,331.4800","-1,969,908.2600","-2,089,456.4800","-2,194,916.9700","-2,359,605.3300","-2,503,780.2900",...,-178.2700,-131.4400,-93.2200,-63.2400,-40.8600,-25.1800,-14.7900,-8.2800,-4.4300,-2.2700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2173,20210630,209,ZOVZ,0.0000,"-1,449,399.9300","-1,596,502.1700","-1,804,554.5300","-1,951,461.1300","-2,080,571.0200","-2,222,607.9000",...,-90.3100,-59.6000,-37.4700,-22.4700,-12.8600,-7.0300,-3.6800,-1.8500,-0.8900,-0.4200
2174,20210630,209,ZPFA,0.0000,"-85,606,409.7100","-88,077,306.9600","-91,337,031.3000","-94,402,963.6500","-96,715,359.5200","-98,885,913.8700",...,"-1,571.6200","-1,247.0400",-973.2000,-743.0700,-551.5000,-395.1600,-271.8400,-179.0800,-112.5900,-67.5300
2175,20210630,209,ZPKS,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2176,20210630,209,ZPPK,0.0000,"1,483,775.2600","1,512,772.3500","-88,425.8900","-170,871.5100","-191,017.2300","-209,944.3600",...,-12.5600,-6.3000,-3.0700,-1.4500,-0.6600,-0.3000,-0.1300,-0.0600,-0.0200,-0.0100


#### Kasstromen premiebetalingen

In [18]:
df_kasstromenPremies = df_kasstromen.copy()
df_kasstromenPremies.iloc[:, 4:104] = df_kasstromenPremies.iloc[:, 4:104].multiply(df_kasstromenPremies['CFP'], axis = "index")
df_kasstromenPremies = df_kasstromenPremies.groupby(['datum', 'risicoType', 'modelpunt']).sum().reset_index()
df_kasstromenPremies = df_kasstromenPremies.drop(columns = ['CF', 'BV', 'CFU', 'CFC', 'CFP'])
df_kasstromenPremies = df_kasstromenPremies.reset_index(drop=True)
df_kasstromenPremies
#df_kasstromenPremies.to_excel('output2.xlsx')

,datum,risicoType,modelpunt,0,12,24,36,48,60,72,...,1080,1092,1104,1116,1128,1140,1152,1164,1176,1188
0,20210630,201,ANWH,0.0000,"10,170,028.7200","745,233.1200","100,680.3900","87,944.4900","80,949.1200","60,921.7400",...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,20210630,201,EABA,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,20210630,201,EABB,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,20210630,201,EABC,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,20210630,201,EABD,0.0000,"2,824.4400","2,821.1100","2,811.4300","2,794.7100","2,773.0200","2,744.3600",...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2173,20210630,209,ZOVZ,0.0000,"283,365.7200","287,729.1400","291,303.6200","294,408.9700","296,841.0400","298,828.5000",...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2174,20210630,209,ZPFA,0.0000,"12,655,009.8800","13,249,707.5800","12,126,029.0100","10,993,796.0800","10,844,766.4800","10,678,228.2300",...,"1,054.0300",831.8300,646.3100,491.8900,364.4100,260.9400,179.4300,118.2100,74.3500,44.5800
2175,20210630,209,ZPKS,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2176,20210630,209,ZPPK,0.0000,"1,713,649.0300","1,763,547.6500","126,065.7300",0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


#### Kasstromen uitkeringen

In [19]:
df_kasstromenUitkeringen = df_kasstromen.copy()
df_kasstromenUitkeringen.iloc[:, 4:104] = df_kasstromenUitkeringen.iloc[:, 4:104].multiply(df_kasstromenUitkeringen['CFU'], axis = "index")
df_kasstromenUitkeringen = df_kasstromenUitkeringen.groupby(['datum', 'risicoType', 'modelpunt']).sum().reset_index()
df_kasstromenUitkeringen = df_kasstromenUitkeringen.drop(columns = ['CF', 'BV', 'CFU', 'CFC', 'CFP'])
df_kasstromenUitkeringen = df_kasstromenUitkeringen.reset_index(drop=True)
df_kasstromenUitkeringen

,datum,risicoType,modelpunt,0,12,24,36,48,60,72,...,1080,1092,1104,1116,1128,1140,1152,1164,1176,1188
0,20210630,201,ANWH,0.0000,"-495,470.0300","-881,910.5300","-917,706.2100","-876,525.3100","-821,989.8600","-762,636.3500",...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,20210630,201,EABA,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,20210630,201,EABB,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,20210630,201,EABC,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,20210630,201,EABD,0.0000,"-1,767,362.7600","-1,873,373.5800","-1,990,282.2900","-2,093,038.4100","-2,254,939.1700","-2,396,294.3300",...,-0.0700,-0.0300,-0.0100,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2173,20210630,209,ZOVZ,0.0000,"-1,664,399.7000","-1,814,226.5700","-2,024,154.0200","-2,172,443.0800","-2,302,213.1900","-2,444,434.2600",...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2174,20210630,209,ZPFA,0.0000,"-97,069,393.5700","-100,116,088.1400","-102,232,939.1500","-104,146,091.3800","-106,288,575.4400","-108,271,799.9000",...,-0.1400,-0.0500,-0.0200,-0.0100,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2175,20210630,209,ZPKS,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2176,20210630,209,ZPPK,0.0000,"-89,849.0600","-108,076.8800","-125,785.5800","-137,829.1300","-157,161.1600","-175,262.7600",...,-3.7100,-1.6900,-0.7500,-0.3200,-0.1300,-0.0600,-0.0200,-0.0100,0.0000,0.0000


#### Kasstromen kosten

In [20]:
df_kasstromenKosten = df_kasstromen.copy()
df_kasstromenKosten.iloc[:, 4:104] = df_kasstromenKosten.iloc[:, 4:104].multiply(df_kasstromenKosten['CFC'], axis = "index")
df_kasstromenKosten = df_kasstromenKosten.groupby(['datum', 'risicoType', 'modelpunt']).sum().reset_index()
df_kasstromenKosten = df_kasstromenKosten.drop(columns = ['CF', 'BV', 'CFU', 'CFC', 'CFP'])
df_kasstromenKosten = df_kasstromenKosten.reset_index(drop=True)
df_kasstromenKosten

,datum,risicoType,modelpunt,0,12,24,36,48,60,72,...,1080,1092,1104,1116,1128,1140,1152,1164,1176,1188
0,20210630,201,ANWH,0.0000,"-797,287.1300","-366,090.5100","-6,498.0000","-5,949.2400","-5,512.1100","-5,036.1900",...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,20210630,201,EABA,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,20210630,201,EABB,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,20210630,201,EABC,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,20210630,201,EABD,0.0000,"-96,793.1600","-99,355.7900","-101,985.6200","-104,673.2700","-107,439.1800","-110,230.3200",...,-178.2000,-131.4100,-93.2100,-63.2400,-40.8600,-25.1800,-14.7900,-8.2800,-4.4300,-2.2700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2173,20210630,209,ZOVZ,0.0000,"-68,365.9500","-70,004.7400","-71,704.1300","-73,427.0200","-75,198.8700","-77,002.1400",...,-90.3100,-59.6000,-37.4700,-22.4700,-12.8600,-7.0300,-3.6800,-1.8500,-0.8900,-0.4200
2174,20210630,209,ZPFA,0.0000,"-1,192,026.0200","-1,210,926.4000","-1,230,121.1600","-1,250,668.3500","-1,271,550.5600","-1,292,342.2000",...,"-2,625.5100","-2,078.8200","-1,619.4900","-1,234.9500",-915.9100,-656.1000,-451.2700,-297.2900,-186.9400,-112.1100
2175,20210630,209,ZPKS,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2176,20210630,209,ZPPK,0.0000,"-140,024.7100","-142,698.4200","-88,706.0400","-33,042.3800","-33,856.0700","-34,681.6000",...,-8.8500,-4.6100,-2.3200,-1.1300,-0.5300,-0.2400,-0.1100,-0.0500,-0.0200,-0.0100


#### Timing kasstromen

In [21]:
df_timingIncome = df_kasstromen.copy()
df_timingIncome = df_timingIncome.loc[df_timingIncome['VAR_NAME'] == 'TIMING_INCOME']
df_timingIncome = df_timingIncome.drop(columns = ['VAR_NAME', 'CF', 'BV', 'CFU', 'CFC', 'CFP'])
df_timingIncome = df_timingIncome.reset_index(drop=True)

df_timingOutgo = df_kasstromen.copy()
df_timingOutgo = df_timingOutgo.loc[df_timingOutgo['VAR_NAME'] == 'TIMING_INCOME']
df_timingOutgo = df_timingOutgo.drop(columns = ['VAR_NAME', 'CF', 'BV', 'CFU', 'CFC', 'CFP'])
df_timingOutgo = df_timingOutgo.reset_index(drop=True)
df_timingOutgo

,datum,risicoType,modelpunt,0,12,24,36,48,60,72,...,1080,1092,1104,1116,1128,1140,1152,1164,1176,1188
0,20210630,201,IAAN,0.0000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000
1,20210630,201,IAAW,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000
2,20210630,201,IABE,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000
3,20210630,201,IADS,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000
4,20210630,201,IAFN,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2173,20210630,209,EARL,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2174,20210630,209,EARM,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2175,20210630,209,EART,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2176,20210630,209,EARV,0.0000,0.9900,0.9600,0.9300,0.9000,0.8800,0.8400,...,0.6500,0.6600,0.6600,0.6600,0.6600,0.0000,0.0000,0.0000,0.0000,0.0000


### Projecties actuariële grootheden
Voor het berekenen van de meerwaarde spaarlossen en voor de berekening van de risk drivers zijn projecties van een aantal grootheden per modelpunt nodig. Voor de meerwaarde spaarlossen is de jaarlijkse oprenting van de voorziening nodig (FN_INTTOEVVP) en voor de risk drivers andere imports. De data voor de meerwaarde spaarlossen is nodig voor zowel het basis scenario als voor de geschokte vtr scenario's. Data voor de risk drivers is alleen voor het basis scenario nodig.

#### Risk drivers

In [22]:
df_driver201InputLeven = pd.read_excel(r"databestanden SSC/Run input/2021Q2_RUN_201_INPUT_Leven (datadrivers).xlsx")
df_driver201InputLeven['risicoType'] = '201'
df_driver201InputPensioenen = pd.read_excel(r"databestanden SSC/Run input/2021Q2_RUN_201_INPUT_Pensioen (datadrivers).xlsx")
df_driver201InputPensioenen['risicoType'] = '201'

dfList = [df_driver201InputLeven, df_driver201InputPensioenen]
df_driverInput = pd.concat(dfList)
df_driverInput = df_driverInput.drop(columns = '!3')
rowList = ['FN_RISPREM', 'FN_KOSTDEK', 'FN_AANTALLYF', 'FN_AANTALPV', 'FN_PPREM_BRU', 'FN_VVPCONS', 'FN_VVPRISPOL', 'FN_PREM_IV', 'FN_U_AFKOOP', 'FN_AANTALPB']
df_driverInput = df_driverInput[df_driverInput[' VAR_NAME'].isin(rowList)]
df_driverInput = df_driverInput.reset_index(drop=True)
df_driverInput['datum'] = datumKwartaal
datum_col = df_driverInput.pop('datum')
df_driverInput.insert(0, 'datum', datum_col)
risico_col = df_driverInput.pop('risicoType')
df_driverInput.insert(1, 'risicoType', risico_col)
df_driverInput.rename(columns={' LIAB_NAME': 'modelpunt', ' VAR_NAME': 'variabele'}, inplace=True)
df_driverInput

,datum,risicoType,modelpunt,variabele,0,12,24,36,48,60,...,1080,1092,1104,1116,1128,1140,1152,1164,1176,1188
0,20210630,201,IAAN,FN_PPREM_BRU,0.0000,"344,943.0200","309,316.6500","277,577.2100","248,671.0500","225,476.2900",...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,20210630,201,IAAN,FN_VVPCONS,"24,763,560.7800","24,830,586.1000","24,781,392.3700","24,676,687.7300","24,519,225.7900","24,310,764.1000",...,604.3000,325.9200,172.1800,89.6600,47.1000,24.7900,13.0600,7.3100,4.0100,1.9700
2,20210630,201,IAAN,FN_U_AFKOOP,0.0000,"125,506.1500","124,533.2400","123,334.9500","122,252.8100","121,251.7200",...,4.1600,2.2700,1.2100,0.6300,0.3300,0.1700,0.0900,0.0500,0.0300,0.0100
3,20210630,201,IAAN,FN_AANTALPV,"11,940.4600","11,888.3400","11,783.6600","11,661.3600","11,502.6300","11,314.9200",...,0.3300,0.2000,0.1200,0.0700,0.0500,0.0300,0.0200,0.0100,0.0100,0.0000
4,20210630,201,IAAN,FN_AANTALPB,"3,317.5200","3,153.6600","2,827.3100","2,519.3500","2,247.4600","2,003.8400",...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2415,20210630,201,EAPF,FN_AANTALLYF,0.0000,7.3800,14.0700,25.5100,37.8800,47.3600,...,0.0300,0.0200,0.0100,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2416,20210630,201,EAPF,FN_KOSTDEK,0.0000,"80,940.5300","82,265.4400","86,576.9600","89,758.3500","93,115.9200",...,8.7800,4.8900,2.6600,1.4100,0.7200,0.3600,0.1700,0.0800,0.0400,0.0200
2417,20210630,201,EAPF,FN_PREM_IV,0.0000,"1,198.7200",0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2418,20210630,201,EAPF,FN_VVPRISPOL,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


#### Meerwaarde spaarlossen

In [23]:
df_spaarlos201Input = pd.read_csv(r"databestanden SSC/Run input/2021Q2_RUN_201_INPUT_Leven.csv",sep = ";",decimal = '.')
df_spaarlos201Input['risicoType'] = '201'
df_spaarlos202Input = pd.read_csv(r"databestanden SSC/Run input/2021Q2_RUN_202_INPUT_Leven.csv",sep = ";",decimal = '.')
df_spaarlos202Input['risicoType'] = '202'
df_spaarlos203Input = pd.read_csv(r"databestanden SSC/Run input/2021Q2_RUN_203_INPUT_Leven.csv",sep = ";",decimal = '.')
df_spaarlos203Input['risicoType'] = '203'
df_spaarlos204Input = pd.read_csv(r"databestanden SSC/Run input/2021Q2_RUN_204_INPUT_Leven.csv",sep = ";",decimal = '.')
df_spaarlos204Input['risicoType'] = '204'
df_spaarlos205Input = pd.read_csv(r"databestanden SSC/Run input/2021Q2_RUN_205_INPUT_Leven.csv",sep = ";",decimal = '.')
df_spaarlos205Input['risicoType'] = '205'
df_spaarlos206Input = pd.read_csv(r"databestanden SSC/Run input/2021Q2_RUN_206_INPUT_Leven.csv",sep = ";",decimal = '.')
df_spaarlos206Input['risicoType'] = '206'
df_spaarlos209Input = pd.read_csv(r"databestanden SSC/Run input/2021Q2_RUN_209_INPUT_Leven.csv",sep = ";",decimal = '.')
df_spaarlos209Input['risicoType'] = '209'
dfList = [df_spaarlos201Input, df_spaarlos202Input, df_spaarlos203Input, df_spaarlos204Input, df_spaarlos205Input, df_spaarlos206Input, df_spaarlos209Input]
df_spaarlosInput = pd.concat(dfList)
df_spaarlosInput = df_spaarlosInput.drop(columns = '!3')
rowList = ['FN_INTTOEVVP']
df_spaarlosInput = df_spaarlosInput[df_spaarlosInput[' VAR_NAME'].isin(rowList)]
df_spaarlosInput = df_spaarlosInput.reset_index(drop=True)
df_spaarlosInput['datum'] = datumKwartaal
datum_col = df_spaarlosInput.pop('datum')
df_spaarlosInput.insert(0, 'datum', datum_col)
risico_col = df_spaarlosInput.pop('risicoType')
df_spaarlosInput.insert(1, 'risicoType', risico_col)
df_spaarlosInput.rename(columns={' LIAB_NAME': 'modelpunt', ' VAR_NAME': 'variabele'}, inplace=True)
df_spaarlosInput

,datum,risicoType,modelpunt,variabele,0,12,24,36,48,60,...,1080,1092,1104,1116,1128,1140,1152,1164,1176,1188
0,20210630,201,IAAN,FN_INTTOEVVP,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,20210630,201,IAAW,FN_INTTOEVVP,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,20210630,201,IABE,FN_INTTOEVVP,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,20210630,201,IADS,FN_INTTOEVVP,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,20210630,201,IAFN,FN_INTTOEVVP,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,20210630,209,FRGR,FN_INTTOEVVP,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1151,20210630,209,FRKL,FN_INTTOEVVP,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1152,20210630,209,FRKR,FN_INTTOEVVP,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1153,20210630,209,UVPF,FN_INTTOEVVP,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


### Rentecurves per kwartaal en actueel

In [24]:
df_curves = {}
df_curves['InclusiefVA'] = pd.read_excel(r"curvebestanden/curvesLiabilities.xlsx", decimal = '.')
df_curves['ExclusiefVA'] = pd.read_excel(r"curvebestanden/curvesRiskMargin.xlsx", decimal = '.')
df_curves['Spaarlossen'] = pd.read_excel(r"curvebestanden/curvesAssets.xlsx", decimal = '.')

df_curves['InclusiefVA']['datum'] = datumKwartaal # in dit notebook dezelfde als actuele datum
df_curves['ExclusiefVA']['datum'] = datumKwartaal # in dit notebook dezelfde als actuele datum
df_curves['Spaarlossen']['datum'] = datumKwartaal # in dit notebook dezelfde als actuele datum

datum_col = df_curves['InclusiefVA'].pop('datum')
df_curves['InclusiefVA'].insert(0, 'datum', datum_col)

datum_col = df_curves['ExclusiefVA'].pop('datum')
df_curves['ExclusiefVA'].insert(0, 'datum', datum_col)

datum_col = df_curves['Spaarlossen'].pop('datum')
df_curves['Spaarlossen'].insert(0, 'datum', datum_col)

pd.options.display.float_format = '{:,.4f}'.format
df_curves['InclusiefVA']
#df_curves['ExclusiefVA']
#df_curves['Spaarlossen']
#df_curves['ExclusiefVA'].to_excel('output.xlsx')

,datum,Jaar,Currency,FairValue,swap.cra.zero.va.down,swap.cra.zero.va.up,SII_basis,SII_Yield_Curve_down,SII_Yield_Curve_up,SII_basis.EQUITY_TYPE_1,...,swap.cra-6m.zero.lip-100.asw200.up-asw200,swap.cra-6m.zero.lip-100.asw200_Currency_Down,swap.cra-6m.zero.lip-100.asw200_Currency_Up,swap.cra-6m.zero.plus100bp.lip-100.asw200,swap.cra-6m.zero.min100bp.lip-100.asw200,swap.cra.zero.va-ratio50.sw310,swap.cra.zero.va-ratio50.sw310.down310,swap.cra.zero.va-ratio50.sw310.up310,swap.cra.zero.va-ratio50.sw310_Currency_Down,swap.cra.zero.va-ratio50.sw310_Currency_Up
0,20210630,1,AUD,0.0001,0.0000,0.0101,0.0001,0.0000,0.0101,0.0001,...,0.0110,0.0010,0.0010,0.0110,-0.0090,0.0001,0.0000,0.0101,0.0001,0.0001
1,20210630,2,AUD,0.0020,0.0007,0.0120,0.0020,0.0007,0.0120,0.0020,...,0.0129,0.0029,0.0029,0.0129,-0.0071,0.0020,0.0007,0.0120,0.0020,0.0020
2,20210630,3,AUD,0.0041,0.0018,0.0141,0.0041,0.0018,0.0141,0.0041,...,0.0150,0.0050,0.0050,0.0150,-0.0050,0.0041,0.0018,0.0141,0.0041,0.0041
3,20210630,4,AUD,0.0062,0.0031,0.0162,0.0062,0.0031,0.0162,0.0062,...,0.0171,0.0071,0.0071,0.0171,-0.0029,0.0062,0.0031,0.0162,0.0062,0.0062
4,20210630,5,AUD,0.0082,0.0044,0.0182,0.0082,0.0044,0.0182,0.0082,...,0.0191,0.0091,0.0091,0.0191,-0.0009,0.0082,0.0044,0.0182,0.0082,0.0082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,20210630,96,ZAR,0.0972,0.0777,0.1166,0.0639,0.0511,0.0767,0.0639,...,0.0491,0.0391,0.0391,0.0418,0.0364,0.0453,0.0363,0.0553,0.0453,0.0453
396,20210630,97,ZAR,0.0972,0.0777,0.1166,0.0638,0.0510,0.0765,0.0638,...,0.0489,0.0389,0.0389,0.0416,0.0362,0.0452,0.0362,0.0552,0.0452,0.0452
397,20210630,98,ZAR,0.0972,0.0777,0.1166,0.0637,0.0510,0.0764,0.0637,...,0.0487,0.0387,0.0387,0.0414,0.0361,0.0450,0.0360,0.0550,0.0450,0.0450
398,20210630,99,ZAR,0.0972,0.0777,0.1166,0.0636,0.0509,0.0763,0.0636,...,0.0485,0.0385,0.0385,0.0412,0.0359,0.0449,0.0359,0.0549,0.0449,0.0449


### LACTP waarden

In [25]:
df_LACTP = pd.read_excel(r"databestanden SSC/LACTP.xlsx")
df_LACTP

,datum,risicoType,LACTP
0,20210630,202,0.0000
1,20210630,203,"73,267,385.4846"
2,20210630,204,0.0000
3,20210630,205,"-9,764,452.8854"
4,20210630,206,"-9,764,452.8854"
5,20210630,207,"-9,764,452.8854"
6,20210630,208,"23,170,581.3511"
7,20210630,209,0.0000


## Documentatie
In dit document staat de methode voor het berekenen van de risicomarge voor de Leven verplichtingen beschreven. Voor het bepalen van de risicomarge is de berekening van de waarde van de verekeringsverplichtingen nodig alsook de berekening van de onderliggende verzekeringstechnische risico's. De methode voor het berekenen van deze waarde en risico's is impliciet onderdeel van deze documentatie.

De volgende verzekeringstechnische risico's worden onderscheiden.

|risicoType|betekenis|
|:---|:---|
|201 | Basis|
|202 | Kortleven risico|
|203 | Langleven risico|
|204 | Arbeidsongeschiktheid risico|
|205 | Verval risico 'down'|
|206 | Verval risico 'up'|
|207 | Verval risico 'mass'|
|208 | Kosten risico|
|209 | Catastrofe risico|
|210 | Operationeel risico| 

De documentatie bestaat uit de volgende onderdelen:
- Verzekeringstechnische risico's 202, 203, 204, 205, 206, 208 en 209 voor nominale en unit linked verplichtingen
- Mass lapse risico (207) voor nominale en unit linked verplichtingen
- Operationeel risico (210) voor nominale en unit linked verplichtingen
- LACTP
- Risico drivers
- Risicomarge

### Verzekeringstechnische risico's 202, 203, 204, 205, 206, 208 en 209
Het verzekeringstechnisch risico (VTR) van type $m$ (= 202, 203, 204, 205, 206, 208 of 209) in geval van rentecurve $i$ voor modelpunt $p$ wordt bij aanvang (tijdstip 0) berekend als het positieve verschil tussen de fair value in het verzekeringstechnische risico scenario $m$ en het verzekeringstechnische basis scenario (201). In geval van een aan het modelpunt gekoppelde spaarlos, wordt gecorrigeerd voor het verschil in de meerwaarde van de spaarlos in beide scenario's.

$$
SCR^{m}_{i,0} = \sum_{p}SCR^{m}_{p,i,0}
$$
$$
SCR^m_{p,i,0} = max\left(\bar{LV}^m_{p,i}-\bar{LV}^{201}_{p,i}-\left(MW^m_{p,i_s}-MW^{201}_{p,i_s}\right),0\right)
$$
waarbij,
$$
\bar{LV}^x_{p,i}= fvc^x_{p,i}\cdot LV^x_{p,i}+ HV_p
$$
met,
$$
lvc^x_{p,i} = 1+\frac{V^x_p-LV^x_{p,0}}{LV^x_{p,i}}
$$, 


|variabele|betekenis|
|:---|:---|
|$$SCR^{m}_{i,0}$$| totaal VTR type $m$ bij rentecurve $i$ in jaar $0$, voor $m$ = 202, 203, 204, 205, 206, 208 of 209|
|$SCR^m_{p,i,0}$| VTR type $m$ bij rentecurve $i$ van modelpunt $p$ in jaar $0$, voor $m$ = 202, 203, 204, 205, 206, 208 of 209|
|$\bar{LV}^x_{p,i}$| liability value van modelpunt $p$ bij rentecurve $i$ in jaar $0$ waarbij is gecorrigeerd voor ijking met de waarde zoals berekend door SSC bij de basis rentecurve alsmede voor de handcorrectie, voor $x$ = 201, 202, 203, 204, 205, 206, 208 of 209|
|$LV^x_{p,i}$| liability value van modelpunt $p$ bij rentecurve $i$ in jaar $0$, voor $x$ = 201, 202, 203, 204, 205, 206, 208 of 209|
|$MW^x_{p,i_s}$| meerwaarde van de spaarlos behorende bij modelpunt $p$ bij rentecurve $i_s$ in jaar $0$, voor $x$ = 201, 202, 203, 204, 205, 206, 208 of 209 (alleen nominaal)|
|$lvc^x_{p,i}$| correctiefactor op de liability value van modelpunt $p$, opdat bij de basis rentecurve $i=0$ wordt aangesloten op de waarde van het modelpunt zoals berekend door SSC ($V$), voor $x$ = 201, 202, 203, 204, 205, 206, 208 of 209|
|$V^x_p$| waarde van het modelpunt $p$ zoals berekend door SSC bij de basis rentecurve, voor $x$ = 201, 202, 203, 204, 205, 206, 208 of 209|
|$HV_p$| handcorrectie op de waarde van het modelpunt $p$, aangeleverd door SSC (alleen nominaal)|


#### Nominaal
$$
LV^x_{p,i} = -\left(\sum^T_{t=1}\left(CP^x_{p,t}\cdot c_{p,\tau^{in}(t)} \cdot D_{i^{c_p},\tau^{in}(t)}+\left(CU^x_{p,t}+CC^x_{p,t}\right)\cdot c_{p,\tau^{out}(t)} \cdot D_{i^{c_p},\tau^{out}(t)}\right) - SB^x_{p,i}\right) \cdot \frac{1}{c_p}
$$
met,
$$
SB^x_{p,i} =
\begin{cases}
    bb\cdot b_p \cdot \sum^T_{t=1}BV^x_{p,t}\cdot c_{p,\tau^{out}(t)}\cdot D_{i^{c_p},\tau^{out}(t)} \hspace{2cm} & \text{if } x=208 \\
    0 & \text{otherwise}
\end{cases}
$$
en,
$$
MW^x_{p,i_s} =
\begin{cases}
    mwc^x_p \cdot \left(\sum^T_{t=1}\left(BV^x_{p,t-1}-BV^x_{p,t}+R^x_{p,t}\right)\cdot D_{i_s,\tau(t)} - BV^x_{p,0}\right)\hspace{2cm} & \text{if } x=201, 202, 203, 204, 205, 206 \text{ of } 209 \\
    MW^{201}_{p,i_s} & \text{if } x=208
\end{cases}
$$
met,
$$
mwc^x_p = \frac{S^x_p}{MW^x_{p,0_s}}
$$


|variabele|betekenis|
|:---|:---|
|$$CP^x_{p,t}$$| cashflow van de premies behorende bij modelpunt $p$ in jaar $t$, voor $x$ = 201, 202, 203, 204, 205, 206, 208 of 209|
|$CC^x_{p,t}$| cashflow van de kosten behorende bij modelpunt $p$ in jaar $t$, voor $x$ = 201, 202, 203, 204, 205, 206, 208 of 209|
|$CU^x_{p,t}$| cashflow van de uitkeringen behorende bij modelpunt $p$ in jaar $t$, voor $x$ = 201, 202, 203, 204, 205, 206, 208 of 209|
|$c_p$| actuele currency exchange rate behorende bij modelpunt $p$|
|$c_{p,\tau^{in}(t)}$| currency exchange rate behorende bij modelpunt $p$ met looptijd $\tau$ van de inkomende kasstromen in jaar $t$|
|$c_{p,\tau^{out}(t)}$| currency exchange rate behorende bij modelpunt $p$ met looptijd $\tau$ van de uitgaande kasstromen in jaar $t$|
|$D_{i^{c_p},\tau^{in}(t)}$| discountfactor van rentecurve $i$ in de valuta $c$ van modelpunt $p$ met looptijd $\tau$ van de inkomende kasstromen in jaar $t$|
|$D_{i^{c_p},\tau^{out}(t)}$| discountfactor van rentecurve $i$ in de valuta $c$ van modelpunt $p$ met looptijd $\tau$ van de uitgaande kasstromen in jaar $t$|
|$D_{i_s,\tau(t)}$| discountfactor van spaarlos rentecurve $i_s$ (in de valuta EUR) met looptijd $\tau$ in jaar $t$|
|$SB^x_{p,i}$| geschokte waarde van de beleggingskosten behorende bij modelpunt $p$ bij rentecurve $i$|
|$bb$| beleggingskosten bijtelling percentage (default gelijk aan 10%)|
|$b_p$| beleggingskosten percentage behorende bij modelpunt $p$|
|$BV^x_{p,t}$| boekwaarde van de voorziening van modelpunt $p$ in jaar $t$, voor $x$ = 201, 202, 203, 204, 205, 206 of 209||
|$R^{201}_{p,t}$| oprenting van de boekwaarde voorziening van modelpunt $p$ in jaar $t$. Uitgangspunt is dat informatie hierover alleen voor 201 wordt verstrekt door SSC|
|$mwc^x_p$| correctiefactor op de meerwaarde van de spaarlos behorende bij modelpunt $p$, opdat bij de basis rentecurve $i=0$ wordt aangesloten op de meerwaarde van de spaarlos zoals berekend door SSC ($S$), voor $x$ = 201, 202, 203, 204, 205, 206 of 209|
|$S^x_p$| meerwaarde van de spaarlos behorende bij modelpunt $p$ bij de basis rentecurve, zoals berekend door SSC, voor $x$ = 201, 202, 203, 204, 205, 206 of 209|

#### Unit linked
Alle unit linked polissen luiden in euro's. De kasstromen en discountfactoren hoeven daarom niet te worden gecorrigeerd voor valutakoersen.

$$
LV^x_{p,i} = BV^x_{p,i,0}-TP^x_{p,i} - SB^x_{p,i}
$$
met,
$$
BV^x_{p,i,0} = BV^x_{p,0}\cdot \left(1-d_p\cdot fi_p\cdot \left( r_{i,d_p}-r^Q_{0,d_p}\right) \right)
$$
en met,
$$
TP^x_{p,i} = \sum^T_{t=1}P^x_{p,i,t} \cdot D_{i,\tau^{out}(t)}
$$
met,
$$
P^x_{p,i,t} = \left( CT^x_{p,t} + \left( BV^x_{p,t-1}-BV^x_{p,t}\right) + BV^x_{p,t-1} \cdot f^Q_{0,t-1} \right) \cdot pc^x_{p,i,t}
$$
waarbij,
$$
pc^x_{p,i,t} =
\begin{cases}
     min\left(100, max\left(-100,\frac{BV^x_{p,i,t-1}}{BV^x_{p,t-1}} \right) \right) \hspace{2cm} & \text{if } BV^x_{p,t-1}=0 \\
    0 & \text{otherwise}
\end{cases}
$$
met,
$$
BV^x_{p,i,t} = BV^x_{p,i,t-1}+CP^x_{p,t}+CC^x_{p,t}+CU^x_{p,t}\cdot pc^x_{p,i,t} + BV^x_{p,i,t-1}\cdot f_{i,t-1} - P^x_{p,i,t} 
$$


|variabele|betekenis|
|:---|:---|
|$$TP^x_{p,i}$$| contant gemaakte total profit behorende bij modelpunt $p$ bij rentecurve $i$, voor $x$ = 201, 202, 203, 204, 205, 206, 208 of 209|
|$P^x_{p,i,t}$| profit in het jaar $t$ behorende bij modelpunt $p$ bij rentecurve $i$, voor $x$ = 201, 202, 203, 204, 205, 206, 208 of 209|
|$D_{i,\tau^{out}(t)}$| discountfactor van rentecurve $i$ (in de valuta EUR) met looptijd $\tau$ in jaar $t$|
|$BV^x_{p,i,t}$| boekwaarde van de voorziening van modelpunt $p$ bij rentecurve $i$ in jaar $t$|
|$d_p$| duratie van de fixed income portefeuille behorende bij modelpunt $p$ (default gelijk aan 5)|
|$fi_p$| percentage fixed income beleggingen in de asset mix behorende bij modelpunt $p$|
|$r_{i,d_p}$| rente met looptijd gelijk aan de duratie van modelpunt $p$ uit de rentecurve $i$|
|$r^Q_{0,d_p}$| rente met looptijd gelijk aan de duratie van modelpunt $p$ uit de basis rentecurve per afgelopen kwartaal|
|$D_{i,\tau^{out}(t)}$|discountfactor van rentecurve $i$ (in de valuta EUR) met looptijd $\tau$ van de uitgaande kasstromen in jaar $t$|
|$CT^x_{p,t}$|cashflows totaal behorende bij modelpunt $p$ in jaar $t$, voor $x$  = 201, 202, 203, 204, 205, 206, 208 of 209|
|$f^Q_{0,t-1}$| 1 jaars forward rente van de basis rentecurve per afgelopen kwartaal in jaar $t-1$|
|$pc^x_{p,i,t}$|correctiefactor op de profit behorende bij modelpunt $p$ in jaar $t$, opdat wordt gecorrigeerd voor de schok in fondswaarde agv rentecurve $i$, voor $x$ = 201, 202, 203, 204, 205, 206 of 209|
|$f_{i,t-1}$| 1 jaars forward rente van de rentecurve $i$ in jaar $t-1$|


### Mass lapse risico (207)
$$
SCR^{207}_{i,0} = \sum_{p}SCR^{207}_{p,i,0}
$$
$$
SCR^{207}_{p,i,0} = a_p \cdot max\left(w_p \cdot \left(\bar{LV}^{207}_{p,i}-\bar{LV}^{201}_{p,i}\right),0\right)
$$
waarbij,
$$
\bar{LV}^{207}_{p,i}= lvc^{207}_{p,i}\cdot LV^{207}_{p,i}
$$
met,
$$
lvc^{207}_{p,i} = 1+\frac{V^{207}_p-LV^{207}_{p,0}}{LV^{207}_{p,i}}
$$ 

|variabele|betekenis|
|:---|:---|
|$$SCR^{207}_{i,0}$$| totaal mass lapse risico bij rentecurve $i$ in jaar $0$|
|$SCR^{207}_{p,i,0}$| mass lapse risico bij rentecurve $i$ van modelpunt $p$ in jaar $0$|
|$a_p$| afkoop indicator behorende bij modelpunt $p$, i.e. gelijk aan 1 indien afkoopbaar en gelijk aan 0 indien niet afkoopbaar|
|$w_p$| weging van mass lapse risico behorende bij modelpunt $p$, i.e. 40% indien individueel en 70% indien collectief|
|$\bar{LV}^{207}_{p,i}$| liability value van modelpunt $p$ igv mass lapse bij rentecurve $i$ in jaar $0$ waarbij is gecorrigeerd voor ijking met de waarde zoals berekend door SSC bij de basis rentecurve|
|$LV^{207}_{p,i}$| liability value van modelpunt $p$ igv masss lapse bij rentecurve $i$ in jaar $0$|
|$lvc^{207}_{p,i}$| correctiefactor op de liability value van modelpunt $p$ igv mass lapse, opdat bij de basis rentecurve $i=0$ wordt aangesloten op de waarde van het modelpunt igv mass lapse zoals berekend door SSC|
|$V^{207}_p$| waarde van het modelpunt $p$ igv mass lapse; afgeleid van data van SSC en de methode verschilt voor uitvaart, individueel overig, collectief en unit linked|

#### Uitvaart
$$
LV^{207}_{p,i} = AV_p + p_p \cdot PV^{207}_{p,i}
$$
met
$$
PV^{207}_{p,i} = -\left(\sum^T_{t=1}\left(CP^{207}_{p,t}\cdot c_{p,\tau^{in}(t)} \cdot D_{i^{c_p},\tau^{in}(t)}+\left(CU^{207}_{p,t}+CC^{207}_{p,t}\right)\cdot c_{p,\tau^{out}(t)} \cdot D_{i^{c_p},\tau^{out}(t)}\right)\right) \cdot \frac{1}{c_p}
$$
en,
$$
V^{207}_p = AV_p + p_p \cdot PV_p  
$$

|variabele|betekenis|
|:---|:---|
|$$AV_p$$ | afkoop vloer behorende bij modelpunt $p$, berekend door SSC|
|$p_p$| premievrijmaak indicator behorende bij modelpunt $p$, i.e. gelijk aan 1 indien afkoopmethode = 2 en gelijk aan 0 anders|
|$PV^{207}_{p,i}$| premievrijmaak vloer behorende bij modelpunt $p$ igv mass lapse bij rentecurve $i$ in jaar 0|
|$PV_p$| premievrijmaak vloer behorende bij modelpunt $p$, berekend door SSC|

#### Individueel exclusief Uitvaart
$$
LV^{207}_{p,i} = AV_p + MW^{201}_{p,i_s}
$$
en,
$$
V^{207}_p = AV_p + S^{201}_p  
$$

#### Collectief
$$
LV^{207}_{p,i} = BV^{207}_{p,0} + HV^{207}_{p}
$$
en,
$$
V^{207}_p = BV^{207}_{p,0} + HV^{207}_{p}
$$

|variabele|betekenis|
|:---|:---|
|$HV^{207}_p$| handcorrectie op de mass lapse waarde van het modelpunt $p$, aangeleverd door SSC|

#### Unit linked
$$
LV^{207}_{p,i} = AV_p -  BV^{207}_{p,0}\cdot d_p\cdot fi_p\cdot \left( r_{i,d_p}-r^Q_{0,d_p}\right) 
$$
en,
$$
V^{207}_p = AV_p
$$

### Verzekeringstechnische risico's na LACTP
Op de hiervoor berekende verzekeringstechnische risico's wordt vervolgens de loss absorbing capital of technical provisions (LACTP) in mindering gebracht. Als input wordt de totale LACTP behorende bij de verschillende risicotypes van SSC ontvangen. Er geldt:

$$
SCR^m_{i,0} = SCR^m_{i,0} - LACTP^m
$$

De totale LACTP per risicotype wordt pro rata de waarde van de verzekeringstechnische risico's omgerekend naar LACTP per modelpunt (per risicotype). Toedeling van de LACTP aan de modelpunten is nodig, omdat voor de berekening van de risicomarge de verzeringstechnische risico's (na LACTP) worden geprojecteerd per modelpunt.

$$
SCR^m_{i,0,p} = SCR^m_{i,0,p} - LACTP^m_{p,i}
$$
met,
$$
LACTP^m_{p,i} = LACTP^m \cdot \frac{SCR^m_{i,0,p}}{SCR^m_{i,0}}
$$

|variabele|betekenis|
|:---|:---|
|$$LACTP^m$$| totale LACTP behorende bij risicotype $m$ in jaar $0$, voor $m$ = 202, 203, 204, 205, 206, 207, 208 en 209|
|$LACTP^m_{p,i}$| LACDTP behorende bij modelpunt $p$ en risicotype $m$ en rentecurve $i$|

### Operationeel risico (210)

$$
SCR^{210}_{i,0} = SCR^{210,nom}_{i,0} + SCR^{210,ul}_{i,0}
$$
met,
$$
SCR^{210,nom}_{i,0} = \sum_{p \in nominaal}SCR^{210,nom}_{p,i,0}
$$
en,
$$
SCR^{210,ul}_{i,0} = \sum_{p \in unit linked}SCR^{210,ul}_{p,i,0}
$$

|variabele|betekenis|
|:---|:---|
|$$SCR^{210}_{i,0}$$| totaal operationeel risico bij rentecurve $i$ in jaar $0$|
|$SCR^{210,nom}_{i,0}$| totaal operationeel risico van de nominale verplichtingen bij rentecurve $i$ in jaar $0$|
|$SCR^{210,ul}_{i,0}$| totaal operationeel risico van de unit linked verplichtingen bij rentecurve $i$ in jaar $0$|
|$SCR^{210,nom}_{p,i,0}$ | operationeel risico bij rentecurve $i$ van nominaal modelpunt $p$ in jaar $0$|
|$SCR^{210,ul}_{p,i,0}$ | operationeel risico bij rentecurve $i$ van unit linked modelpunt $p$ in jaar $0$|

#### Nominaal

$$
SCR^{210,nom}_{p,i,0} =
\begin{cases}
    OV_i \cdot \frac{abs(LV^{201}_{p,i}\cdot lvc^{210}_{p,i})}{\sum_{p \in nominaal}abs(LV^{201}_{p,i}\cdot lvc^{201}_{p,i})} \hspace{2cm} & \text{if } OV_i > OP \\
    0.04 \cdot PB_p  & \text{otherwise}
\end{cases}
$$
met,
$$
OV_i = 0.0045 \cdot\sum_{p \in nominaal}LV^{201}_{p,i}\cdot lvc^{210}_{p,i}
$$
en met,
$$
lvc^{210}_{p,i} = 1+\frac{\left(V^{201}_p+HV_p \right)-LV^{201}_{p,0}}{LV^{201}_{p,i}}
$$
en met,
$$
OP = 0.04 \cdot \sum_{p \in nominaal} BP_p
$$

|variabele|betekenis|
|:---|:---|
|$$OV_i$$| totaal operationeel risico mbt de nominale voorzieningen bij rentecurve $i$ in jaar $0$|
|$OP$| totaal operationeel risico mbt de bruto premies in jaar $0$|
|$PB$| bruto premie behorende bij modelpunt $p$ in jaar $0$|
|$lvc^{210}_{p,i}$| correctiefactor op de liability value van modelpunt $p$ igv operationeel risico, opdat bij de basis rentecurve $i=0$ wordt aangesloten op de waarde van het modelpunt plus handcorrectie zoals berekend door SSC|


#### Unit linked

$$
SCR^{210,ul}_{p,i,0} = SCR^{210,ul,L}_{p,0} + SCR^{210,ul,P}_{p,0}
$$
met,
$$
SCR^{210,ul,x}_{p,0} = 0.25 \cdot \frac{nP^{ul,x}_{p}}{\sum_{p \in ul, p \in x} nP^{ul,x}_{p}} \cdot K^x \cdot kc^x
$$
met,
$$
kc^x = \frac{\sum_{p \in ul, p \in x} nP^{ul,x}_{p}}{\sum_{p \in ul, p \in x} nP^{ul,x}_{p} + \sum_{p \in ul/nom, p \in x} nP^{ul/nom,x}_{p}}
$$

|variabele|betekenis|
|:---|:---|
|$$SCR^{210,ul,L}_{p,0}$$| operationeel risico van unit linked leven modelpunt $p$ in jaar $0$|
|$SCR^{210,ul,P}_{p,0}$| operationeel risico van unit linked pensioen modelpunt $p$ in jaar $0$|
|$nP^{ul,x}_{p}$| aantal polissen behorende bij unit linked modelpunt $p$, voor $x$ gelijk aan $L$ (leven) of $P$ pensioen|
|$nP^{ul,x}_{p}$| aantal polissen behorende bij unit linked modelpunt $p$ die zal worden overgezet naar nominaal modelpunt, voor $x$ gelijk aan $L$ (leven) of $P$ pensioen|
|$K^x$| operationele kosten voor unit linked aangeleverd door SSC, voor $x$ gelijk aan $L$ (leven) of $P$ pensioen|
|$kc^x$| correctie factor op operationele unit linked kosten, opdat de kosten worden gecorrigeerd (naar beneden worden bijgesteld) om rekening te houden met modelpunten die van unit linked worden overgezet naar nominaal, voor $x$ gelijk aan $L$ (leven) of $P$ pensioen|

### Risicodrivers

$$
RD^m_{p,i,t} = 
\begin{cases}
    \frac{DD^m_{p,i,t+1}}{DD^m_{p,i,1}}\hspace{2cm} & \text{if } m <> 202, 209 \\
    \frac{DD^m_{p,i,t+1}}{DD^m_{p,i,1}}\cdot \frac{100-t}{100} & m = 202 \\
    \frac{SCR^{202}_{p,i,t}+SCR^{204}_{p,i,t}}{SCR^{202}_{p,i,0}+SCR^{204}_{p,i,0}} & m = 209
\end{cases}
$$

|variabele|betekenis|
|:---|:---|
|$$RD^m_{p,i,t}$$| risicodriver van modelpunt $p$ bij rentecurve $i$ in jaar $t$|
|$$DD^m_{p,i,t}$$| datadriver van modelpunt $p$ bij rentecurve $i$ in jaar $t$|


#### Kortleven risico (202)
$$
DD^{202}_{p,i,t} = max\left(RP_{p,t},0\right)+KD_{p,t}
$$

|variabele|betekenis|
|:---|:---|
|$$RP_{p,t}$$| risicopremie behorende bij modelpunt $p$ in jaar $t$ (FN_RISPREM)|
|$KD_{p,t}$| kostendekking van modelpunt $p$ in jaar $t$ (FN_KOSTDEK)|

#### Langleven risico (203)
$$
DD^{203}_{p,i,t} = 
\begin{cases}
    nLF_{p,t}\hspace{2cm} & \text{if } m_p = \text{# Lijfrenteposten} \\
    nPV_{p,t}\hspace{2cm} & \text{if } m_p = \text{# PremieVrije(PUP)} \\
    BP_{p,t} + LC_{p,t} & \text{anders}
\end{cases}
$$
met,
$$
LC_{p,t} = L_{p,t-1} + LP_{p,t-1} \cdot iC_p
$$

|variabele|betekenis|
|:---|:---|
|$$nLF_{p,t}$$| aantal lijfrente polissen behorende bij modelpunt $p$ in jaar $t$ (FN_AANTALLYF)|
|$nPV_{p,t}$| aantal premievrije polissen behorende bij modelpunt $p$ in jaar $t$ (FN_AANTALPV)|
|$BP_{p,t}$| bruto premie behorende bij modelpunt $p$ in jaar $t$ (FN_PPREM_BRU)|
|$LC_{p,t}$| vvp na correctie behorende bij modelpunt $p$ in jaar $t$ (FN_VVPCONS_na_correctie)|
|$L_{p,t}$| vvp behorende bij modelpunt $p$ in jaar $t$ (FN_VVPCONS)|
|$LP_{p,t}$| vvp risico polishouders behorende bij modelpunt $p$ in jaar $t$ (FN_VVPRISPOL)|
|$iC_p$| indicator correctie risico driver behorende bij modelpunt $p$, (1 of 0) (Ind_VVP_Riskdriver_Corrigeren)|
|$m_p$| methode risicodriver behorende bij modelpunt $p$|

#### Arbeidsongeschiktheid risico (204)
$$
DD^{204}_{p,i,t} = IP_{p,t}
$$

|variabele|betekenis|
|:---|:---|
|$$IP_{p,t}$$| (invaliditeits?)premie behorende bij modelpunt $p$ in jaar $t$ (FN_PREM_IV)|

#### Lapse risico (205, 206 en 207)
$$
DD^{m}_{p,i,t} = 
\begin{cases}
    \left(\frac{DD^{m}_{p,i,t+1}}{1+f_{i^{c_p},t}}\cdot c_{p,t}+BP_{p,t}\cdot c_{p,t}\right)\cdot\frac{1}{c_{p,t}} \hspace{2cm} & \text{if } m_p = \text{CW(PR)} \\
    \left(\frac{DD^{m}_{p,i,t+1}}{1+f_{i^{c_p},t}}\cdot c_{p,t}+BP_{p,t}\cdot c_{p,t}\right)\cdot\frac{1}{c_{p,t}}\cdot\frac{99-t}{t} \hspace{2cm} & \text{if } m_p = \text{CW(PR) daling} \\
    \left(\frac{DD^{m}_{p,i,t+1}}{1+f_{i^{c_p},t}}\cdot c_{p,t}+\left(RP_{p,t}+KD_{p,t}\right)\cdot c_{p,t}\right)\cdot\frac{1}{c_{p,t}} \hspace{2cm} & \text{if } m_p = \text{CW(KD+PR)} \\
    UA_{p,t} & \text{anders}
\end{cases}
$$

|variabele|betekenis|
|:---|:---|
|$$f_{i^{c_p},t}$$| 1 jaars forward rente behorende bij rentecurve $i$ en de valuta $c$ waarin modelpunt $p$ is genoteerd in jaar $t$|
|$c_{p,t}$| currency exchange rate behorende bij modelpunt $p$ in jaar $t$|
|$UA_{p,t}$| afkoop uitkering(?) behorende bij modelpunt $p$ in jaar $t$|

#### Kosten risico (208)
$$
DD^{208}_{p,i,t} = \frac{\left(1+i\right)^t}{\left(1+i+is\right)^t} \cdot\left(\frac{DD^{208}_{p,i,t+1}}{1+f_{i^{c_p},t}}\cdot c_{p,t}+RE^{208}_{p,t}\cdot c_{p,t}\right)\cdot\frac{1}{c_{p,t}}-\left(\frac{DD^{201}_{p,i,t+1}}{1+f_{i^{c_p},t}}\cdot c_{p,t}+RE^{201}_{p,t}\cdot c_{p,t}\right)\cdot\frac{1}{c_{p,t}}
$$
met,
$$
i = \frac{is \cdot RE^{201}_{p,t} \cdot \left( 1+ks\right)}{\left(RE^{208}_{p,t}-RE^{201}_{p,t}\cdot\left(1+ks \right) \right)}-1
$$

|variabele|betekenis|
|:---|:---|
|$$i$$| inflatie|
|$is$| inflatie schok volgens Solvency II (0.01)|
|$ks$| kosten schok volgens Solvency II (0.1)|
|$RE^m_{p,t}$| renewal(?) kosten behorende bij modelpunt $p$ in jaar $t$ voor risicotype $m$, waarbij $m$ = 201 en 208|

#### Operationeel risico (210)
$$
DD^{210}_{p,i,t} = 
\begin{cases}
    nPV_{p,t} + nPB_{p,t} + nLF_{p,t}\hspace{2cm} & \text{if } p = \text{unit linked} \\
    max\left(pd\cdot BP_{p,t}, ld\cdot LC_{p,t}\right) & \text{anders}
\end{cases}
$$

|variabele|betekenis|
|:---|:---|
|$$nPB_{p,t}$$| aantal premie betalende polissen behorende bij modelpunt  𝑝  in jaar  𝑡  (FN_AANTALPB)|
|$pd$| premie risico driver (0.03)|
|$ld$| liability value risico driver (0.003)|

### Risicomarge
De risicomarge in geval van rentecurve $i$ wordt als volgt berekend. 

$$
RM_i = CoC \cdot \sum^T_{t=0}{SCRT_{i,t} \cdot D_{i,t}}
$$
met
$$
SCRT_{i,t} = \sqrt{\sum_{x \in M}\sum_{y \in M}SCR^x_{i,t}\cdot SCR^y_{i,t} \cdot \rho_{xy}} + SCR^{210}_{i,t}
$$
met
$$
M = \left\{ 202, 203, 204, lapse, 208, revision, 209 \right\}
$$
waarbij
$$
\begin{align}
SCR^{lapse}_{i,t} &= max\left(SCR^{205}_{i,t}, SCR^{206}_{i,t}, SCR^{207}_{i,t}\right) \\
SCR^{revision}_{i,t} &= 0
\end{align}
$$

Het totale verzekeringstechnische risico van type $m$ in geval van rentecurve $i$ in jaar $t$ is gelijk aan de optelsom van de betreffende risico's voor ieder modelpunt $p$.
$$
SCR^m_{i,t} = \sum_{p} SCR^m_{p,i,t}
$$
met
$$
SCR^m_{p,i,t} = SCR^m_{p,i,0} \cdot RD^m_{p,i,t}
$$

|variabele|betekenis|
|:---|:---|
|$$RM_i$$| risicomarge in geval van rentecurve $i$|
|$CoC$| cost of capital (0.06)|
|$D_{i,t}$| discountfactor van rentecurve $i$ voor jaar $t$|
|$SCR^m_{i,t}$| risico bij rentecurve $i$ in jaat $t$ voor de risico's $m$, $m$ = 202, 203, 204, 205, 206, 207, 208, 209, 210 en revision|



## Modellering

In [26]:
def addTwoDataFrames(df1, df2): # df2 = dataframe with the most columns
    modelpunten = df1['modelpunt']
    df1 = df1.drop(['modelpunt'], axis = 1)
    df2 = df2.drop(['modelpunt'], axis = 1)
    columnnames = df2.columns
    np_vector = df1.to_numpy()
    np_matrix = df2.to_numpy()
    np = np_vector + np_matrix
    df = pd.DataFrame(np, columns = columnnames)
    df.insert(0, 'modelpunt', modelpunten)
    return df

In [27]:
def multiplyTwoDataFrames(df1, df2): # df1 = dataframe with the most columns
    modelpunten = df1['modelpunt']
    df1 = df1.drop(['modelpunt'], axis = 1)
    df2 = df2.drop(['modelpunt'], axis = 1)
    columnnames = df1.columns
    np_matrix = df1.to_numpy() # could also be a matrix
    np_vector = df2.to_numpy()
    np = np_matrix * np_vector
    df = pd.DataFrame(np, columns = columnnames)
    df.insert(0, 'modelpunt', modelpunten)
    return df    

In [28]:
def multiplyDataFrameWithFloat(df, x):
    modelpunten = df['modelpunt']
    df = df.drop(['modelpunt'], axis = 1).astype(float)
    df = df * x
    df.insert(0, 'modelpunt', modelpunten)
    return df 

In [29]:
def convertNegativesToZeros(df):
    modelpunten = df['modelpunt']
    df = df.drop(['modelpunt'], axis = 1)
    df[df < 0] = 0
    df.insert(0, 'modelpunt', modelpunten)
    return df

In [30]:
def berekenTimingDelta(datumKwartaal, datum):    
    datumKwartaalDate = datetime.strptime(str(datumKwartaal), '%Y%m%d')
    datumDate = datetime.strptime(str(datum), '%Y%m%d')
    timingDelta = (datumDate - datumKwartaalDate).days/365
    return timingDelta

In [31]:
def berekenForwardRente(t, renteCurve):
    if t == 1:
        forwardRente = renteCurve[0]
    else:
        forwardRente = (1+renteCurve[t-1])**t / (1+renteCurve[t-2])**(t-1) - 1
    return forwardRente

In [32]:
def berekenForwardRentes(renteCurve):
    forwardRentes = []
    for t in range(0,99):
        forwardRente = berekenForwardRente(t+1, renteCurve)
        forwardRentes += [forwardRente]
    return forwardRentes

In [33]:
def berekenLiabilityValueHandCorrectie():
    list_LVHandCorrectie = []
    for modelpunt in modelpuntNamen:
        projectieBoekwaardeVoorziening = df_projectieBoekwaardeVoorziening.loc[(df_projectieBoekwaardeVoorziening['datum'] == datumKwartaal) & (df_projectieBoekwaardeVoorziening['risicoType'] == '201') & (df_projectieBoekwaardeVoorziening['modelpunt'] == modelpunt), 0:].values[0][0]
        if projectieBoekwaardeVoorziening == 0:
            list_LVHandCorrectie += [0]
        else:
            LVHandCorrectie = df_waardenVoorzieningenGamma.loc[(df_waardenVoorzieningenGamma['datum'] == datumKwartaal) & (df_waardenVoorzieningenGamma['modelpunt'] == modelpunt), '201Hand'].values[0]
            list_LVHandCorrectie += [LVHandCorrectie]

    df_LVHandCorrectie['LVHandCorrectie'] = list_LVHandCorrectie

In [34]:
def berekenRentesTiming(valuta, datum, timing, renteCurve, curveType):
    rentes = df_curves[curveType].loc[(df_curves[curveType]['datum'] == datum) & (df_curves[curveType]['Currency'] == valuta), renteCurve].values
    jaren = range(0,100)
    rentesTiming = np.interp(timing, jaren, rentes)
    return rentesTiming

In [35]:
def berekenExchangeRate(valuta):
    if valuta == 'AUD':
        exchangeRate = EURAUD
    elif valuta == 'USD':
        exchangeRate = EURUSD
    elif valuta == 'ZAR':
        exchangeRate = EURZAR
    elif valuta == 'EUR':
        exchangeRate = 1
    else:
        print('Valuta onbekend')
    return exchangeRate

In [36]:
def berekenToekomstigeValutas(valuta, exchangeRate, datum, timing, renteCurve, curveType):
    list_Valutas = []  
    if valuta != 'EUR':
        rentesTimingForeign = berekenRentesTiming(valuta, datum, timing, renteCurve, curveType)
        rentesTimingEUR = berekenRentesTiming('EUR', datum, timing, renteCurve, curveType)
        for t in range(0,100):
            list_Valutas += [exchangeRate * (1 + rentesTimingForeign[t])**timing[t] / (1 + rentesTimingEUR[t])**timing[t]]
    else:
        list_Valutas = [1 for t in range(0,100)]
    return list_Valutas 
        

In [37]:
def berekenLiabilityValueNominaal(timingDelta, datum, risicoType, purpose, renteCurve, modelpunt):
    marktWaarde = 0
    if purpose == 'SCR':
        curveType = 'InclusiefVA'
    else:
        curveType = 'ExclusiefVA'
    
    valuta = df_valuta.loc[df_valuta['modelpunt'] == modelpunt, 'currency'].values[0]
    exchangeRate = berekenExchangeRate(valuta)
    
    if (risicoType != '207') | (df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'nominaalUitvaart'].values[0] == 1):
        kasstromenPremies = df_kasstromenPremies.loc[(df_kasstromenPremies['datum'] == datumKwartaal) & (df_kasstromenPremies['risicoType'] == risicoType) & (df_kasstromenPremies['modelpunt'] == modelpunt), 0:].values[0]
        kasstromenUitkeringen = df_kasstromenUitkeringen.loc[(df_kasstromenUitkeringen['datum'] == datumKwartaal) & (df_kasstromenUitkeringen['risicoType'] == risicoType) & (df_kasstromenUitkeringen['modelpunt'] == modelpunt), 0:].values[0]
        kasstromenKosten = df_kasstromenKosten.loc[(df_kasstromenKosten['datum'] == datumKwartaal) & (df_kasstromenKosten['risicoType'] == risicoType) & (df_kasstromenKosten['modelpunt'] == modelpunt), 0:].values[0]
        timingIncome = df_timingIncome.loc[(df_timingIncome['datum'] == datumKwartaal) & (df_timingIncome['risicoType'] == risicoType) & (df_timingIncome['modelpunt'] == modelpunt), 0:].values[0]
        timingOutgo = df_timingOutgo.loc[(df_timingOutgo['datum'] == datumKwartaal) & (df_timingOutgo['risicoType'] == risicoType) & (df_timingOutgo['modelpunt'] == modelpunt), 0:].values[0]    

        jaren = range(0,100)
        jarenTimingIncome = np.array(jaren) - (1-timingIncome) - timingDelta
        jarenTimingOutgo = np.array(jaren) - (1-timingOutgo) - timingDelta
        rentesIncome = berekenRentesTiming(valuta, datum, jarenTimingIncome, renteCurve, curveType)
        rentesOutgo = berekenRentesTiming(valuta, datum, jarenTimingOutgo, renteCurve, curveType)

        valutasIncome = berekenToekomstigeValutas(valuta, exchangeRate, datum, jarenTimingIncome, renteCurve, curveType)
        valutasOutgo = berekenToekomstigeValutas(valuta, exchangeRate, datum, jarenTimingOutgo, renteCurve, curveType)

        if risicoType != '207':
            if risicoType == '208':
                projectieBoekwaardeVoorziening = df_projectieBoekwaardeVoorziening.loc[(df_projectieBoekwaardeVoorziening['datum'] == datumKwartaal) & (df_projectieBoekwaardeVoorziening['risicoType'] == risicoType) & (df_projectieBoekwaardeVoorziening['modelpunt'] == modelpunt), 0:].values[0]
                beleggingskostenPercentage = df_beleggingskostenPercentages.loc[(df_beleggingskostenPercentages['datum'] == datumKwartaal) & (df_beleggingskostenPercentages['modelpunt'] == modelpunt), 'beleggingskostenPercentage'].values[0] / 100 

            marktWaardeBeleggingsKosten = 0
            for t in range(1, 100):
                marktWaarde += -(kasstromenPremies[t] * valutasIncome[t] / (1+rentesIncome[t])**jarenTimingIncome[t] + (kasstromenUitkeringen[t]+kasstromenKosten[t]) * valutasOutgo[t] / (1+rentesOutgo[t-1])**jarenTimingOutgo[t])

                # bereken marktWaardeBeleggingsKosten
                if risicoType == '208':
                    marktWaardeBeleggingsKosten = marktWaardeBeleggingsKosten + beleggingskostenPercentage*projectieBoekwaardeVoorziening[t] * valutasOutgo[t] / (1+rentesOutgo[t-1])**jarenTimingOutgo[t] 

            #print(modelpunt, 'Nominaal', risicoType, renteCurve, -marktWaarde)
            marktWaarde += beleggingskostenBijtellingPercentage * marktWaardeBeleggingsKosten
            marktWaarde = marktWaarde / exchangeRate
            
    if (risicoType == '207'):           
        if df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'nominaalUitvaart'].values[0] == 1:
            AV = df_vloerenGamma.loc[df_vloerenGamma['modelpunt'] == modelpunt, 'afkoopVloer'].values[0]
            if df_vloerenGamma.loc[df_vloerenGamma['modelpunt'] == modelpunt, 'afkoopMethode'].values[0] == 2:
                pvi = 1
            else:
                pvi = 0
            PV = 0
            for t in range(1, 100): 
                PV += -(kasstromenPremies[t] * valutasIncome[t] / (1+rentesIncome[t])**jarenTimingIncome[t] + (kasstromenUitkeringen[t]+kasstromenKosten[t]) * valutasOutgo[t] / (1+rentesOutgo[t-1])**jarenTimingOutgo[t])
            PV = PV / exchangeRate
            marktWaarde = AV + pvi * PV
        elif df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'nominaalRest'].values[0] == 1:
            AV = df_vloerenGamma.loc[df_vloerenGamma['modelpunt'] == modelpunt, 'afkoopVloer'].values[0]
            MWSPL = berekenMeerwaardeSpaarlos(timingDelta, datum, '201', renteCurve, modelpunt)
            marktWaarde = AV + MWSPL
        elif df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'nominaalCollectief'].values[0] == 1:
            BV = df_projectieBoekwaardeVoorziening.loc[(df_projectieBoekwaardeVoorziening['datum'] == datumKwartaal) & (df_projectieBoekwaardeVoorziening['risicoType'] == risicoType) & (df_projectieBoekwaardeVoorziening['modelpunt'] == modelpunt), 0].values[0] 
            LVHandCorrectie = df_waardenVoorzieningenGamma.loc[(df_waardenVoorzieningenGamma['datum'] == datumKwartaal) & (df_waardenVoorzieningenGamma['modelpunt'] == modelpunt), '207Hand'].values[0]
            marktWaarde = BV + LVHandCorrectie
        else:
            marktWaarde = 0

    return marktWaarde   

In [38]:
def berekenLiabilityValueUnitLinked(timingDelta, datum, risicoType, purpose, renteCurve, modelpunt):
    marktWaarde = 0
    if purpose == 'SCR':
        curveType = 'InclusiefVA'
    else:
        curveType = 'ExclusiefVA' 
        
    projectieBoekwaardeVoorziening = df_projectieBoekwaardeVoorziening.loc[(df_projectieBoekwaardeVoorziening['datum'] == datumKwartaal) & (df_projectieBoekwaardeVoorziening['risicoType'] == risicoType) & (df_projectieBoekwaardeVoorziening['modelpunt'] == modelpunt), 0:].values[0]
    projectieBoekwaardeVoorzieningRisicoTypeBasis = df_projectieBoekwaardeVoorziening.loc[(df_projectieBoekwaardeVoorziening['datum'] == datumKwartaal) & (df_projectieBoekwaardeVoorziening['risicoType'] == '201') & (df_projectieBoekwaardeVoorziening['modelpunt'] == modelpunt), 0:].values[0]
    kasstromenTotaal = df_kasstromenTotaal.loc[(df_kasstromenTotaal['datum'] == datumKwartaal) & (df_kasstromenTotaal['risicoType'] == risicoType) & (df_kasstromenTotaal['modelpunt'] == modelpunt), 0:].values[0]
    kasstromenPremies = df_kasstromenPremies.loc[(df_kasstromenPremies['datum'] == datumKwartaal) & (df_kasstromenPremies['risicoType'] == risicoType) & (df_kasstromenPremies['modelpunt'] == modelpunt), 0:].values[0]
    kasstromenUitkeringen = df_kasstromenUitkeringen.loc[(df_kasstromenUitkeringen['datum'] == datumKwartaal) & (df_kasstromenUitkeringen['risicoType'] == risicoType) & (df_kasstromenUitkeringen['modelpunt'] == modelpunt), 0:].values[0]
    kasstromenKosten = df_kasstromenKosten.loc[(df_kasstromenKosten['datum'] == datumKwartaal) & (df_kasstromenKosten['risicoType'] == risicoType) & (df_kasstromenKosten['modelpunt'] == modelpunt), 0:].values[0]
    timingIncome = df_timingIncome.loc[(df_timingIncome['datum'] == datumKwartaal) & (df_timingIncome['risicoType'] == risicoType) & (df_timingIncome['modelpunt'] == modelpunt), 0:].values[0]
    timingOutgo = df_timingOutgo.loc[(df_timingOutgo['datum'] == datumKwartaal) & (df_timingOutgo['risicoType'] == risicoType) & (df_timingOutgo['modelpunt'] == modelpunt), 0:].values[0]    

    valuta = df_valuta.loc[df_valuta['modelpunt'] == modelpunt, 'currency'].values[0]        
    rentesRenteCurveBasisDatumKwartaal = df_curves[curveType].loc[(df_curves[curveType]['datum'] == datumKwartaal) & (df_curves[curveType]['Currency'] == valuta), renteCurveBasis].values
    rentesRenteCurveDatum = df_curves[curveType].loc[(df_curves[curveType]['datum'] == datum) & (df_curves[curveType]['Currency'] == valuta), renteCurve].values
    
    jaren = range(0,100)
    jarenTimingOutgo = np.array(jaren) - (1-timingOutgo) - timingDelta
    rentesOutgo = berekenRentesTiming(valuta, datum, jarenTimingOutgo, renteCurve, curveType)
    
    if risicoType == '208':
        beleggingskostenPercentage = df_beleggingskostenPercentages.loc[(df_beleggingskostenPercentages['datum'] == datumKwartaal) & (df_beleggingskostenPercentages['modelpunt'] == modelpunt), 'beleggingskostenPercentage'].values[0] / 100

    # delta rente om impact op boekwaarde voorziening op t=0 te bepalen
    deltaInterestRate = rentesRenteCurveDatum[durationBonds] - rentesRenteCurveBasisDatumKwartaal[durationBonds] 
    
    # percentage obligaties in de asset mix om impact op boekwaarde voorziening op t=0 te bepalen
    if df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'unitLinkedCollectief'].values[0] == 1:
        percentageBonds = percentageBondsCollectief
    else:
        percentageBonds = percentageBondsIndividueel
        
    # boekwaarde voorziening ná schok op t=0
    boekwaardeVoorzieningRenteCurveDatumSchokBijStart = projectieBoekwaardeVoorziening[0] * durationBonds*percentageBonds*deltaInterestRate
    boekwaardeVoorzieningRenteCurveDatumStart = projectieBoekwaardeVoorziening[0] - boekwaardeVoorzieningRenteCurveDatumSchokBijStart
    boekwaardeVoorzieningRenteCurveDatum = boekwaardeVoorzieningRenteCurveDatumStart
    
    # bereken marktWaarde
    if risicoType != '207':
        totalProfit = 0
        marktWaardeBeleggingsKosten = 0
        for t in range(1, 100):
            if projectieBoekwaardeVoorziening[t-1] == 0:    
                correctieBoekwaardeVoorziening = 0
            else:
                correctieBoekwaardeVoorziening = min(100, max(-100, boekwaardeVoorzieningRenteCurveDatum / projectieBoekwaardeVoorziening[t-1]))

            # bereken profit
            forwardRenteCurveBasisDatumKwartaal = berekenForwardRente(t, rentesRenteCurveBasisDatumKwartaal)
            profit = (kasstromenTotaal[t] + (projectieBoekwaardeVoorziening[t-1] - projectieBoekwaardeVoorziening[t]) + projectieBoekwaardeVoorziening[t-1]*forwardRenteCurveBasisDatumKwartaal)*correctieBoekwaardeVoorziening

            # bereken marktWaardeBeleggingsKosten
            if risicoType == '208':
                marktWaardeBeleggingsKosten = marktWaardeBeleggingsKosten + beleggingskostenPercentage*boekwaardeVoorzieningRenteCurveDatum / (1+rentesOutgo[t-1])**jarenTimingOutgo[t] 

            # herbereken boekwaardeVoorziening en boekwaardeVoorzieningRenteCurveDatum
            forwardRenteCurveDatum = berekenForwardRente(t, rentesRenteCurveDatum) # Vraag aan MavenBlue: Hoe moet de forward rente worden aangepast voor timingOutgo en timingDelta?
            boekwaardeVoorzieningRenteCurveDatum = boekwaardeVoorzieningRenteCurveDatum + kasstromenPremies[t] + kasstromenKosten[t] + kasstromenUitkeringen[t]*correctieBoekwaardeVoorziening + boekwaardeVoorzieningRenteCurveDatum*forwardRenteCurveDatum - profit

            # bereken totalProfit
            totalProfit = totalProfit + profit / (1+rentesOutgo[t-1])**jarenTimingOutgo[t] 

        marktWaarde = boekwaardeVoorzieningRenteCurveDatumStart - totalProfit + beleggingskostenBijtellingPercentage * marktWaardeBeleggingsKosten
    if risicoType == '207':
        AV = df_vloerenGamma.loc[df_vloerenGamma['modelpunt'] == modelpunt, 'afkoopVloer'].values[0]
        marktWaarde = AV - boekwaardeVoorzieningRenteCurveDatumSchokBijStart
    
    return marktWaarde

In [39]:
def berekenLiabilityValueVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves(datum, purpose):
    timingDelta = berekenTimingDelta(datumKwartaal, datum)
    for renteCurve in renteCurves:
        for risicoType in risicoTypes[:9]:
            list_LV = []
            for modelpunt in modelpuntNamen:

                # NOMINAAL
                if df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'nominaal'].values[0] == 1:
                    LV = berekenLiabilityValueNominaal(timingDelta, datum, risicoType, purpose, renteCurve, modelpunt)
                    list_LV += [LV]

                # UNIT LINKED
                else:
                    LV = berekenLiabilityValueUnitLinked(timingDelta, datum, risicoType, purpose, renteCurve, modelpunt)
                    list_LV += [LV]

            df_LV[datum, risicoType, purpose][renteCurve] = list_LV
              

In [40]:
def berekenLiabilityValueVoorBasisRisicoOverAlleModelpuntenVoorAlleRenteCurves(datum):
    df_LVBasis[datum] = multiplyTwoDataFrames(df_LV[datum, '201', 'SCR'], df_correctieLVFactoren['201'])
    df_LVBasis[datum] = addTwoDataFrames(df_LVHandCorrectie, df_LVBasis[datum])
    df_LVBasis[datum] = pd.merge(df_LVBasis[datum], df_productSoorten[['modelpunt', 'unitLinked', 'ULGarantie', 'unitLinkedCollectief', 'nominaal', 'nominaalCollectief']], how = 'left', on = 'modelpunt')
    df_LVBasis[datum].insert(0, 'liabilityType', '')
    df_LVBasis[datum].loc[(df_LVBasis[datum]['unitLinkedCollectief'] == 1) & (df_LVBasis[datum]['ULGarantie'] != 1), 'liabilityType'] = 'ULCollectief'
    df_LVBasis[datum].loc[(df_LVBasis[datum]['unitLinked'] == 1) & (df_LVBasis[datum]['unitLinkedCollectief'] != 1) & (df_LVBasis[datum]['ULGarantie'] != 1), 'liabilityType'] = 'ULIndividueel'
    df_LVBasis[datum].loc[(df_LVBasis[datum]['unitLinkedCollectief'] == 1) & (df_LVBasis[datum]['ULGarantie'] == 1), 'liabilityType'] = 'ULCollectiefGarantie'
    df_LVBasis[datum].loc[(df_LVBasis[datum]['unitLinkedCollectief'] != 1) & (df_LVBasis[datum]['ULGarantie'] == 1), 'liabilityType'] = 'ULIndividueelGarantie'
    df_LVBasis[datum].loc[(df_LVBasis[datum]['nominaalCollectief'] == 1), 'liabilityType'] = 'nominaalCollectief'
    df_LVBasis[datum].loc[(df_LVBasis[datum]['nominaal'] == 1) & (df_LVBasis[datum]['nominaalCollectief'] != 1), 'liabilityType'] = 'nominaalIndividueel'
    df_LVBasis[datum] = df_LVBasis[datum].drop(columns = ['unitLinked', 'unitLinkedCollectief', 'ULGarantie', 'nominaal', 'nominaalCollectief'])
    df_LVBasis[datum] = df_LVBasis[datum].groupby('liabilityType').sum()
    df_total = df_LVBasis[datum].sum(axis=0).to_frame().T
    df_total.index = ['total']
    df_LVBasis[datum] = pd.concat([df_LVBasis[datum],df_total], ignore_index=False)

    #df_LVBasis[datum]

In [41]:
def berekenUnitLinkedProfitVoorBasisRisicoOverAlleModelpuntenVoorAlleRenteCurves(datum):
    df_ULProfit[datum] = multiplyTwoDataFrames(df_LV[datum, '201', 'SCR'], df_correctieLVFactoren['201'])
    df_ULProfit[datum] = pd.merge(df_ULProfit[datum], df_projectieBoekwaardeVoorziening[['modelpunt', 0]].loc[df_projectieBoekwaardeVoorziening['risicoType'] == '201'][['modelpunt', 0]], how = 'left', on = 'modelpunt')
    df_ULProfit[datum].rename(columns={0: 'bookValue'}, inplace=True)
    df_bookValue = df_ULProfit[datum][['modelpunt', 'bookValue']]
    df_ULProfit[datum] = addTwoDataFrames(df_bookValue, -1*df_ULProfit[datum])
    df_ULProfit[datum] = pd.merge(df_ULProfit[datum], df_productSoorten[['modelpunt', 'unitLinked', 'ULGarantie', 'unitLinkedCollectief']], how = 'left', on = 'modelpunt')
    df_ULProfit[datum].drop(df_ULProfit[datum].loc[df_ULProfit[datum]['unitLinked'] != 1].index, inplace=True)
    df_ULProfit[datum].insert(1, 'unitLinkedType', '')
    df_ULProfit[datum].loc[(df_ULProfit[datum]['unitLinkedCollectief'] == 1) & (df_ULProfit[datum]['ULGarantie'] != 1), 'unitLinkedType'] = 'ULCollectief'
    df_ULProfit[datum].loc[(df_ULProfit[datum]['unitLinkedCollectief'] != 1) & (df_ULProfit[datum]['ULGarantie'] != 1), 'unitLinkedType'] = 'ULIndividueel'
    df_ULProfit[datum].loc[(df_ULProfit[datum]['unitLinkedCollectief'] == 1) & (df_ULProfit[datum]['ULGarantie'] == 1), 'unitLinkedType'] = 'ULCollectiefGarantie'
    df_ULProfit[datum].loc[(df_ULProfit[datum]['unitLinkedCollectief'] != 1) & (df_ULProfit[datum]['ULGarantie'] == 1), 'unitLinkedType'] = 'ULIndividueelGarantie'
    df_ULProfit[datum] = df_ULProfit[datum].drop(columns = ['unitLinked', 'unitLinkedCollectief', 'ULGarantie', 'bookValue'])
    df_ULProfit[datum] = df_ULProfit[datum].groupby('unitLinkedType').sum()
    df_total = df_ULProfit[datum].sum(axis=0).to_frame().T
    df_total.index = ['total']
    df_ULProfit[datum] = pd.concat([df_ULProfit[datum],df_total], ignore_index=False)
    
    #df_ULProfit[datum]

In [42]:
def berekenCorrectieLiabilityValueFactorenVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves():
    for risicoType in risicoTypes[:9]:
        for renteCurve in renteCurves:
            list_correctieLVFactoren = []
            for modelpunt in modelpuntNamen:
                LVRenteCurveBasis = df_LV[datumKwartaal, risicoType, 'SCR'].loc[df_LV[datumKwartaal, risicoType, 'SCR']['modelpunt'] == modelpunt, renteCurveBasis].values[0]
                LVRenteCurveBasisRisicoTypeBasis = df_LV[datumKwartaal, '201', 'SCR'].loc[df_LV[datumKwartaal, '201', 'SCR']['modelpunt'] == modelpunt, renteCurveBasis].values[0]
                LVRenteCurve = df_LV[datum, risicoType, 'SCR'].loc[df_LV[datum, risicoType, 'SCR']['modelpunt'] == modelpunt, renteCurve].values[0]
                if risicoType != '207':
                    LVGamma = df_waardenVoorzieningenGamma.loc[(df_waardenVoorzieningenGamma['datum'] == datumKwartaal) & (df_waardenVoorzieningenGamma['modelpunt'] == modelpunt), risicoType].values[0]
                if risicoType == '207':
                    if df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'nominaal'].values[0] == 1:
                        if df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'nominaalUitvaart'].values[0] == 1:
                            AV = df_vloerenGamma.loc[df_vloerenGamma['modelpunt'] == modelpunt, 'afkoopVloer'].values[0]
                            if df_vloerenGamma.loc[df_vloerenGamma['modelpunt'] == modelpunt, 'afkoopMethode'].values[0] == 2:
                                pvi = 1
                            else:
                                pvi = 0
                            PVGamma = df_vloerenGamma.loc[df_vloerenGamma['modelpunt'] == modelpunt, 'premieVrijmaakVloer'].values[0]
                            LVGamma = AV + pvi * PVGamma
                        elif df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'nominaalRest'].values[0] == 1:
                            AV = df_vloerenGamma.loc[df_vloerenGamma['modelpunt'] == modelpunt, 'afkoopVloer'].values[0]
                            meerwaardeSpaarlosGamma = df_meerwaardeSpaarlossenGamma.loc[(df_meerwaardeSpaarlossenGamma['datum'] == datumKwartaal) & (df_meerwaardeSpaarlossenGamma['modelpunt'] == modelpunt), '201'].values[0] 
                            LVGamma = AV + meerwaardeSpaarlosGamma
                        elif df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'nominaalCollectief'].values[0] == 1:
                            BVGamma = df_projectieBoekwaardeVoorziening.loc[(df_projectieBoekwaardeVoorziening['datum'] == datumKwartaal) & (df_projectieBoekwaardeVoorziening['risicoType'] == risicoType) & (df_projectieBoekwaardeVoorziening['modelpunt'] == modelpunt), 0].values[0] 
                            LVHandCorrectieGamma = df_waardenVoorzieningenGamma.loc[(df_waardenVoorzieningenGamma['datum'] == datumKwartaal) & (df_waardenVoorzieningenGamma['modelpunt'] == modelpunt), '207Hand'].values[0]   
                            LVGamma = BVGamma + LVHandCorrectieGamma                                
                        else:
                            LVGamma = LVRenteCurveBasis
                    else: # UNIT LINKED
                        AVGamma = df_vloerenGamma.loc[df_vloerenGamma['modelpunt'] == modelpunt, 'afkoopVloer'].values[0]
                        LVGamma = AVGamma                             

                if (abs(LVRenteCurveBasisRisicoTypeBasis) < 2) | (abs(LVGamma) < 2): # Het kan gebeuren dat een mp waarde 0 heeft bij 201, maar > 0 bij 208 agv beleggingskosten. In dat geval niet corrigeren naar LVGamma
                    correctieLVFactor = 1
                else:
                    correctieLVFactor = 1 + (LVGamma - LVRenteCurveBasis) / LVRenteCurve
                list_correctieLVFactoren += [correctieLVFactor]

            df_correctieLVFactoren[risicoType][renteCurve] = list_correctieLVFactoren

In [43]:
def berekenMeerwaardeSpaarlos(timingDelta, datum, risicoType, renteCurve, modelpunt):
    meerwaardeSpaarlosGamma = df_meerwaardeSpaarlossenGamma.loc[(df_meerwaardeSpaarlossenGamma['datum'] == datumKwartaal) & (df_meerwaardeSpaarlossenGamma['modelpunt'] == modelpunt), risicoType].values[0]
    meerwaardeSpaarlos = 0
    if meerwaardeSpaarlosGamma != 0:
        if (risicoType != '208'):
            projectieBoekwaardeVoorziening = df_projectieBoekwaardeVoorziening.loc[(df_projectieBoekwaardeVoorziening['datum'] == datumKwartaal) & (df_projectieBoekwaardeVoorziening['risicoType'] == risicoType) & (df_projectieBoekwaardeVoorziening['modelpunt'] == modelpunt), 0:].values[0]   
            projectieINTTOEVVP = df_spaarlosInput.loc[(df_spaarlosInput['datum'] == datumKwartaal) & (df_spaarlosInput['risicoType'] == risicoType) & (df_spaarlosInput['modelpunt'] == modelpunt) & (df_spaarlosInput['variabele'] == 'FN_INTTOEVVP'), '0':].values[0] 
            jaren = range(0,100)
            jarenTiming = np.array(jaren) - timingDelta
            rentesTiming = berekenRentesTiming('EUR', datum, jarenTiming, renteCurve, 'Spaarlossen')

            meerwaardeSpaarlos = 0
            for t in range(1, 100):
                meerwaardeSpaarlos += ((projectieBoekwaardeVoorziening[t-1] - projectieBoekwaardeVoorziening[t]) + projectieINTTOEVVP[t]) / (1 + iboxxSpread + rentesTiming[t-1])**jarenTiming[t]
            meerwaardeSpaarlos -= projectieBoekwaardeVoorziening[0]

        if (risicoType == '208'):
            meerwaardeSpaarlos = df_MWSPL[datum, '201'].loc[(df_MWSPL[datum, '201']['modelpunt'] == modelpunt), renteCurve].values[0] 
    return meerwaardeSpaarlos

In [44]:
def berekenMeerwaardeSpaarlossenVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves(datum):
    timingDelta = berekenTimingDelta(datumKwartaal, datum)
    for risicoType in risicoTypes[:9]:
        for renteCurve in renteCurves:
            list_MWSPL = []
            for modelpunt in modelpuntNamen:

                # NOMINAAL individueel
                if (df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'nominaal'].values[0] == 1) & (df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'nominaalCollectief'].values[0] != 1) & (risicoType != '207'):
                    MWSPL = berekenMeerwaardeSpaarlos(timingDelta, datum, risicoType, renteCurve, modelpunt)
                    list_MWSPL += [MWSPL]

                # UNIT LINKED en Collectief en risicoType '207'
                else:
                    list_MWSPL += [0]

            df_MWSPL[datum, risicoType][renteCurve] = list_MWSPL

In [45]:
def berekenCorrectieMeerwaardeSpaarlossenFactorenVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves():
    for risicoType in risicoTypes[:9]:
        list_correctieMWSPLFactoren = []
        for modelpunt in modelpuntNamen:
            MWSPLGamma = df_meerwaardeSpaarlossenGamma.loc[(df_meerwaardeSpaarlossenGamma['datum'] == datumKwartaal) & (df_meerwaardeSpaarlossenGamma['modelpunt'] == modelpunt), risicoType].values[0]
            MWSPLRenteCurveBasis = df_MWSPL[datumKwartaal, risicoType].loc[df_MWSPL[datumKwartaal, risicoType]['modelpunt'] == modelpunt, renteCurveBasis].values[0]
            if risicoType == '207':
                correctieMWSPLFactor = 0
            elif (MWSPLGamma == 0):
                correctieMWSPLFactor = 1
            else:
                correctieMWSPLFactor = MWSPLGamma / MWSPLRenteCurveBasis
            list_correctieMWSPLFactoren += [correctieMWSPLFactor]

        df_correctieMWSPLFactoren[risicoType][renteCurveBasis] = list_correctieMWSPLFactoren  

In [46]:
def berekenSCRAnteLACTPVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves(datum, purpose):
    for risicoType in risicoTypes[1:9]:
        LVBasis = multiplyTwoDataFrames(df_LV[datum, '201', purpose], df_correctieLVFactoren['201'])
        LVBasis = addTwoDataFrames(df_LVHandCorrectie, LVBasis)
        LVRisico = multiplyTwoDataFrames(df_LV[datum, risicoType, purpose], df_correctieLVFactoren[risicoType])
        if risicoType != '207':
            LVRisico = addTwoDataFrames(df_LVHandCorrectie, LVRisico)
            DifferenceLVRisicoLVBasis = addTwoDataFrames(LVRisico, -1*LVBasis)

            MWSPLRisico = multiplyTwoDataFrames(df_MWSPL[datum, risicoType], df_correctieMWSPLFactoren[risicoType]) 
            MWSPLBasis = multiplyTwoDataFrames(df_MWSPL[datum, '201'], df_correctieMWSPLFactoren['201'])
            DifferenceMWSPLRisicoMWSPLBasis = addTwoDataFrames(MWSPLRisico, -1*MWSPLBasis)

            df_SCR[datum, risicoType, purpose] = addTwoDataFrames(DifferenceLVRisicoLVBasis, -1*DifferenceMWSPLRisicoMWSPLBasis)

        if risicoType == '207':
            DifferenceLVRisicoLVBasis = addTwoDataFrames(LVRisico, -1*LVBasis)
            df_SCR[datum, risicoType, purpose] = multiplyTwoDataFrames(DifferenceLVRisicoLVBasis, df_productSoorten.loc[:,['modelpunt', 'weight']])
            df_SCR[datum, risicoType, purpose] = multiplyTwoDataFrames(df_SCR[datum, risicoType, purpose], df_vloerenGamma.loc[:,['modelpunt', 'afkoopIndicator']])
        df_SCR[datum, risicoType, purpose] = convertNegativesToZeros(df_SCR[datum, risicoType, purpose])

In [47]:
def berekenLACTPVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves(datum, purpose):
    for risicoType in risicoTypes[1:9]:
        LACTP = df_LACTP.loc[(df_LACTP['datum'] == datumKwartaal) & (df_LACTP['risicoType'] == int(risicoType)), 'LACTP'].values[0] 
        for renteCurve in renteCurves:
            SCRTotaal = sum(df_SCR[datum, risicoType, purpose][renteCurve])
            list_LACTPMP = []
            for modelpunt in modelpuntNamen:
                SCRMP = df_SCR[datum, risicoType, purpose].loc[df_SCR[datum, risicoType, purpose]['modelpunt'] == modelpunt, renteCurve].values[0]
                if SCRTotaal == 0:
                    LACTPMP = 0
                else:
                    LACTPMP = LACTP * SCRMP / SCRTotaal
                list_LACTPMP += [LACTPMP]

            df_LACTPMP[datum, risicoType, purpose][renteCurve] = list_LACTPMP

In [48]:
def berekenSCRVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves(datum, purpose):
    for risicoType in risicoTypes[1:9]:
        df_SCR[datum, risicoType, purpose] = addTwoDataFrames(df_SCR[datum, risicoType, purpose], -1*df_LACTPMP[datum, risicoType, purpose])

In [49]:
def berekenORPremieNominaal():
    ORPremieNominaal = 0
    for modelpunt in modelpuntNamen:
        if df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'nominaalOperationeelRisico'].values[0] == 1:
            premieBruto = df_driverInput.loc[(df_driverInput['datum'] == datumKwartaal) & (df_driverInput['risicoType'] == '201') & (df_driverInput['modelpunt'] == modelpunt) & (df_driverInput['variabele'] == 'FN_PPREM_BRU'), 12].values[0]
            ORPremieNominaal += percentageOperationeelRisicoPremie * premieBruto
    return ORPremieNominaal

In [50]:
def berekenLVCorrectie(datum, renteCurve, modelpunt):
    LVRenteCurveBasisRisicoTypeBasis = df_LV[datumKwartaal, '201', 'SCR'].loc[df_LV[datumKwartaal, '201', 'SCR']['modelpunt'] == modelpunt, renteCurveBasis].values[0]
    LVRenteCurveRisicoTypeBasis = df_LV[datum, '201', 'SCR'].loc[df_LV[datum, '201', 'SCR']['modelpunt'] == modelpunt, renteCurve].values[0]
    LVGamma = df_waardenVoorzieningenGamma.loc[(df_waardenVoorzieningenGamma['datum'] == datumKwartaal) & (df_waardenVoorzieningenGamma['modelpunt'] == modelpunt), '201'].values[0]
    LVHandCorrectie = df_waardenVoorzieningenGamma.loc[(df_waardenVoorzieningenGamma['datum'] == datumKwartaal) & (df_waardenVoorzieningenGamma['modelpunt'] == modelpunt), '201Hand'].values[0]
    LVRenteCurveRisicoTypeBasis = df_LV[datum, '201', 'SCR'].loc[df_LV[datum, '201', 'SCR']['modelpunt'] == modelpunt, renteCurve].values[0]
    if LVRenteCurveRisicoTypeBasis != 0:
        LVCorrectie = 1 + ((LVGamma + LVHandCorrectie) - LVRenteCurveBasisRisicoTypeBasis) / LVRenteCurveRisicoTypeBasis
    else:
        LVCorrectie = 1
    return LVCorrectie   

In [51]:
def berekenVoorzieningNominaalMP(datum, purpose, renteCurve, modelpunt):
    LVCorrectie = berekenLVCorrectie(datum, renteCurve, modelpunt)
    LVRenteCurveRisicoTypeBasis = df_LV[datum, '201', purpose].loc[df_LV[datum, '201', 'SCR']['modelpunt'] == modelpunt, renteCurve].values[0]
    voorzieningNominaalMP = LVRenteCurveRisicoTypeBasis * LVCorrectie
    return voorzieningNominaalMP

In [52]:
def berekenORVoorzieningNominaal(datum, purpose, renteCurve): 
    voorzieningNominaal = 0
    ORVoorzieningNominaal = 0
    voorzieningNominaalAbsoluut = 0
    for modelpunt in modelpuntNamen:
        if df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'nominaalOperationeelRisico'].values[0] == 1:
            voorzieningNominaalMP = berekenVoorzieningNominaalMP(datum, purpose, renteCurve, modelpunt) 
            voorzieningNominaal += voorzieningNominaalMP
            voorzieningNominaalAbsoluut += abs(voorzieningNominaalMP)
    ORVoorzieningNominaal = percentageOperationeelRisicoVoorziening * voorzieningNominaal      
    return ORVoorzieningNominaal, voorzieningNominaalAbsoluut

In [53]:
def berekenSCROperationeelNominaalVoorModelpunt(datum, purpose, renteCurve, modelpunt, voorzieningNominaalAbsoluut, ORVoorzieningNominaal, ORPremieNominaal):
    if ORVoorzieningNominaal > ORPremieNominaal:
        voorzieningNominaalMP = berekenVoorzieningNominaalMP(datum, purpose, renteCurve, modelpunt)
        SCRoperationeelNominaalMP = ORVoorzieningNominaal * abs(voorzieningNominaalMP) / voorzieningNominaalAbsoluut
    else:
        premieBruto = df_driverInput.loc[(df_driverInput['datum'] == datumKwartaal) & (df_driverInput['risicoType'] == '201') & (df_driverInput['modelpunt'] == modelpunt) & (df_driverInput['variabele'] == 'FN_PPREM_BRU'), 12].values[0]
        SCRoperationeelNominaalMP = percentageOperationeelRisicoPremie * premieBruto
    return SCRoperationeelNominaalMP

In [54]:
def berekenAantalPolissenUnitLinkedOperationeelRisico(individueelOfCollectief, ULNaarNom):
    aantal = 0
    for modelpunt in modelpuntNamen:
        if ULNaarNom == False:
            if (df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'unitLinkedOperationeelRisico'].values[0] == 1) & (df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'productlijn'].values[0] == individueelOfCollectief):
                aantal += df_driverInput.loc[(df_driverInput['datum'] == datumKwartaal) & (df_driverInput['risicoType'] == '201') & (df_driverInput['modelpunt'] == modelpunt) & ((df_driverInput['variabele'] == 'FN_AANTALLYF') | (df_driverInput['variabele'] == 'FN_AANTALPB') | (df_driverInput['variabele'] == 'FN_AANTALPV')), 12].sum()
        else:
            if (df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'unitLinkedNaarNominaalOperationeelRisico'].values[0] == 1)  & (df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'productlijn'].values[0] == individueelOfCollectief):            
                aantal += df_driverInput.loc[(df_driverInput['datum'] == datumKwartaal) & (df_driverInput['risicoType'] == '201') & (df_driverInput['modelpunt'] == modelpunt) & ((df_driverInput['variabele'] == 'FN_AANTALLYF') | (df_driverInput['variabele'] == 'FN_AANTALPB') | (df_driverInput['variabele'] == 'FN_AANTALPV')), 12].sum()
    return aantal

In [55]:
def berekenSCROperationeelVoorAlleModelpuntenVoorAlleRenteCurves(datum, purpose):
    ORPremieNominaal = berekenORPremieNominaal()
    aantalPolissenUnitLinkedOperationeelRisicoLeven = berekenAantalPolissenUnitLinkedOperationeelRisico(2, False)
    aantalPolissenUnitLinkedOperationeelRisicoPensioenen = berekenAantalPolissenUnitLinkedOperationeelRisico(3, False)
    aantalPolissenUnitLinkedNaarNominaalLeven = berekenAantalPolissenUnitLinkedOperationeelRisico(2, True)
    aantalPolissenUnitLinkedNaarNominaalPensioenen = berekenAantalPolissenUnitLinkedOperationeelRisico(3, True)
    if aantalPolissenUnitLinkedOperationeelRisicoLeven + aantalPolissenUnitLinkedNaarNominaalLeven == 0:
        ORKostenULLevenCorrectie = 1
    else:
        ORKostenULLevenCorrectie = aantalPolissenUnitLinkedOperationeelRisicoLeven / (aantalPolissenUnitLinkedOperationeelRisicoLeven + aantalPolissenUnitLinkedNaarNominaalLeven)
    if aantalPolissenUnitLinkedOperationeelRisicoPensioenen + aantalPolissenUnitLinkedNaarNominaalPensioenen == 0:
        ORKostenULPensioenenCorrectie = 1
    else:
        ORKostenULPensioenenCorrectie = aantalPolissenUnitLinkedOperationeelRisicoPensioenen / (aantalPolissenUnitLinkedOperationeelRisicoPensioenen + aantalPolissenUnitLinkedNaarNominaalPensioenen)
    for renteCurve in renteCurves:
        ORVoorzieningNominaal, voorzieningNominaalAbsoluut = berekenORVoorzieningNominaal(datum, purpose, renteCurve)
        list_ORModelpunten = []
        for modelpunt in modelpuntNamen:

            # NOMINAAL
            if df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'nominaalOperationeelRisico'].values[0] == 1:
                ORModelpunt = berekenSCROperationeelNominaalVoorModelpunt(datum, purpose, renteCurve, modelpunt, voorzieningNominaalAbsoluut, ORVoorzieningNominaal, ORPremieNominaal)
                
            # UNIT LINKED
            else:
                aantalPolissenUnitLinkedOperationeelRisicoMP = df_driverInput.loc[(df_driverInput['datum'] == datumKwartaal) & (df_driverInput['risicoType'] == '201') & (df_driverInput['modelpunt'] == modelpunt) & ((df_driverInput['variabele'] == 'FN_AANTALLYF') | (df_driverInput['variabele'] == 'FN_AANTALPB') | (df_driverInput['variabele'] == 'FN_AANTALPV')), 12].sum()
                if df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'productlijn'].values[0] == 2: # Leven
                    ORModelpunt = percentageOperationeelRisicoUnitLinkedKosten * ORKostenULLeven * ORKostenULLevenCorrectie * aantalPolissenUnitLinkedOperationeelRisicoMP / aantalPolissenUnitLinkedOperationeelRisicoLeven
                else: # Pensioenen
                    ORModelpunt = percentageOperationeelRisicoUnitLinkedKosten * ORKostenULPensioenen * ORKostenULPensioenenCorrectie * aantalPolissenUnitLinkedOperationeelRisicoMP / aantalPolissenUnitLinkedOperationeelRisicoPensioenen
            
            list_ORModelpunten += [ORModelpunt]
            #print(renteCurve, modelpunt, ORModelpunt)
        df_SCR[datum, '210', purpose][renteCurve] = list_ORModelpunten


In [56]:
def aggregeerSCRVoorAlleRisicoTypesOverAlleModelpuntenVoorAlleRenteCurves(datum, purpose):
    i = 0
    for risicoType in risicoTypes[1:]:   
        df = df_SCR[datum, risicoType, purpose].sum(axis=0, skipna = True, numeric_only = True)
        df_SCRGeaggregeerd[datum, purpose].iloc[i,1:] = df
        i += 1

In [57]:
def berekenDrivers(datum, modelpunt, methodeRiskDriver, renteCurve, risicoType, baseScenario):
    valuta = df_valuta.loc[df_valuta['modelpunt'] == modelpunt, 'currency'].values[0]
    exchangeRate = berekenExchangeRate(valuta)
    rentesRenteCurve = df_curves['ExclusiefVA'].loc[(df_curves['ExclusiefVA']['datum'] == datum) & (df_curves['ExclusiefVA']['Currency'] == valuta), renteCurve].values
    forwardRentes = berekenForwardRentes(rentesRenteCurve)
    valutas = berekenToekomstigeValutas(valuta, exchangeRate, datum, range(0,100), renteCurve, 'ExclusiefVA')
   
    valuation = True
    if (risicoType == '205') | (risicoType == '206') | (risicoType == '207'):
        if (methodeRiskDriver == 'CW(PR)') | (methodeRiskDriver == 'CW(PR) daling'):
            CFs = df_driverInput.loc[(df_driverInput['datum'] == datumKwartaal) & (df_driverInput['modelpunt'] == modelpunt) & (df_driverInput['variabele'] == 'FN_PPREM_BRU'), 0:].values[0]
        elif (methodeRiskDriver == 'CW(KD+PR)'):
            CFs = df_driverInput.loc[(df_driverInput['datum'] == datumKwartaal) & (df_driverInput['modelpunt'] == modelpunt) & (df_driverInput['variabele'] == 'FN_RISPREM'), 0:].values[0] \
                    + df_driverInput.loc[(df_driverInput['datum'] == datumKwartaal) & (df_driverInput['modelpunt'] == modelpunt) & (df_driverInput['variabele'] == 'FN_KOSTDEK'), 0:].values[0]
        else:
            valuation = False
            CFs = df_driverInput.loc[(df_driverInput['datum'] == datumKwartaal) & (df_driverInput['modelpunt'] == modelpunt) & (df_driverInput['variabele'] == 'FN_U_AFKOOP'), 0:].values[0]       
            
    if (risicoType == '208'):
        if baseScenario == True:
            CFs = df_kasstromen.loc[(df_kasstromen['datum'] == datumKwartaal) & (df_kasstromen['risicoType'] == '201') & (df_kasstromen['modelpunt'] == modelpunt) & (df_kasstromen['VAR_NAME'] == 'REN_EXP'), 0:].values[0]    
        else:
            CFs = df_kasstromen.loc[(df_kasstromen['datum'] == datumKwartaal) & (df_kasstromen['risicoType'] == '208') & (df_kasstromen['modelpunt'] == modelpunt) & (df_kasstromen['VAR_NAME'] == 'REN_EXP'), 0:].values[0]    
            
    driver = 0
    drivers = [0]         
    for t in range(1, 100):
        if valuation == True:
            driver = driver * valutas[98 - t] / (1 + forwardRentes[98 - t]) +  CFs[99 - t] * valutas[98 - t]
            drivers.insert(0, driver / valutas[98 - t]) # KLOPT HET DAT VALUTA PER SALDO GEEN IMPACT HEEFT
        else:
            driver = CFs[99 - t]
            drivers.insert(0, driver)         
    return drivers

In [58]:
def updateRiskDriversMP(lijst, t, driver, driver1, risicoType):
    if driver1 == 0:
        riskDriverMP = 0
    elif (risicoType == '202'):
        riskDriverMP = driver / driver1 * (100 - (t-1)) / 100
    else:
        riskDriverMP = driver / driver1
    lijst += [riskDriverMP] 
    return lijst

In [59]:
def berekenValueNaCorrectie(t, modelpunt, indicatorCorrectie):
    v1 = df_driverInput.loc[(df_driverInput['datum'] == datumKwartaal) & (df_driverInput['modelpunt'] == modelpunt) & (df_driverInput['variabele'] == 'FN_VVPCONS'), 12*(t-1)].values[0]
    v2 = df_driverInput.loc[(df_driverInput['datum'] == datumKwartaal) & (df_driverInput['modelpunt'] == modelpunt) & (df_driverInput['variabele'] == 'FN_VVPRISPOL'), 12*(t-1)].values[0]
    valueNaCorrectie = v1 + v2 * indicatorCorrectie
    return valueNaCorrectie

In [60]:
def berekenInflatie(modelpunt):
    value201 = df_kasstromen.loc[(df_kasstromen['datum'] == datumKwartaal) & (df_kasstromen['risicoType'] == '201') & (df_kasstromen['modelpunt'] == modelpunt) & (df_kasstromen['VAR_NAME'] == 'REN_EXP'), 12].values[0]
    value208 = df_kasstromen.loc[(df_kasstromen['datum'] == datumKwartaal) & (df_kasstromen['risicoType'] == '208') & (df_kasstromen['modelpunt'] == modelpunt) & (df_kasstromen['VAR_NAME'] == 'REN_EXP'), 12].values[0]
    if value208 - value201 * (1 + schokKosten) == 0:
        inflatie = 0
    else:
        inflatie = (schokInflatie * value201 * (1 + schokKosten)) / (value208 - value201 * (1 + schokKosten)) - 1
    return inflatie

In [61]:
def berekenRiskDriversVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves(datum):
    for renteCurve in renteCurves:
        for risicoType in risicoTypes[1:10]:
            for modelpunt in modelpuntNamen:        
                if (risicoType == '202'): 
                    list_riskDriversMP = [1]
                    for t in range(1, 100):
                        value1 = df_driverInput.loc[(df_driverInput['datum'] == datumKwartaal) & (df_driverInput['modelpunt'] == modelpunt) & (df_driverInput['variabele'] == 'FN_RISPREM'), 12*t].values[0]
                        value2 = df_driverInput.loc[(df_driverInput['datum'] == datumKwartaal) & (df_driverInput['modelpunt'] == modelpunt) & (df_driverInput['variabele'] == 'FN_KOSTDEK'), 12*t].values[0]
                        driver = max(value1, 0) + value2
                        if t == 1:
                            driver1 = driver
                        else:
                            list_riskDriversMP = updateRiskDriversMP(list_riskDriversMP, t, driver, driver1, risicoType)
                if (risicoType == '203'): 
                    list_riskDriversMP = [1]
                    indicatorCorrectie = df_methodenRiskDrivers.loc[df_methodenRiskDrivers['modelpunt'] == modelpunt, 'indicatorCorrectie'].values[0]
                    for t in range(1, 100):
                        if df_methodenRiskDrivers.loc[df_methodenRiskDrivers['modelpunt'] == modelpunt, risicoType].values[0] == '# Lijfrenteposten':
                            driver = df_driverInput.loc[(df_driverInput['datum'] == datumKwartaal) & (df_driverInput['modelpunt'] == modelpunt) & (df_driverInput['variabele'] == 'FN_AANTALLYF'), 12*t].values[0]
                        elif df_methodenRiskDrivers.loc[df_methodenRiskDrivers['modelpunt'] == modelpunt, risicoType].values[0] == '# PremieVrije(PUP)': 
                            driver = df_driverInput.loc[(df_driverInput['datum'] == datumKwartaal) & (df_driverInput['modelpunt'] == modelpunt) & (df_driverInput['variabele'] == 'FN_AANTALPV'), 12*t].values[0]
                        else:
                            value1 = df_driverInput.loc[(df_driverInput['datum'] == datumKwartaal) & (df_driverInput['modelpunt'] == modelpunt) & (df_driverInput['variabele'] == 'FN_PPREM_BRU'), 12*t].values[0]
                            value2 = berekenValueNaCorrectie(t, modelpunt, indicatorCorrectie)
                            driver = value1 + value2 
                        if t == 1:
                            driver1 = driver
                        else:
                            list_riskDriversMP = updateRiskDriversMP(list_riskDriversMP, t, driver, driver1, risicoType)
                if (risicoType == '204'):
                    list_riskDriversMP = [1]
                    for t in range(1, 100):
                        driver = df_driverInput.loc[(df_driverInput['datum'] == datumKwartaal) & (df_driverInput['modelpunt'] == modelpunt) & (df_driverInput['variabele'] == 'FN_PREM_IV'), 12*t].values[0]
                        if t == 1:
                            driver1 = driver
                        else:
                            list_riskDriversMP = updateRiskDriversMP(list_riskDriversMP, t, driver, driver1, risicoType)
                if (risicoType == '205') | (risicoType == '206') | (risicoType == '207'):
                    list_riskDriversMP = [1]
                    methodeRiskDriver = df_methodenRiskDrivers.loc[df_methodenRiskDrivers['modelpunt'] == modelpunt, risicoType].values[0]
                    drivers = berekenDrivers(datum, modelpunt, methodeRiskDriver, renteCurve, risicoType, True)    
                    for t in range(1, 100):
                        driver = drivers[t]
                        if methodeRiskDriver == 'CW(PR) daling':
                            driver = driver * (99 - t) / 99 
                        if t == 1:
                            driver1 = driver
                        else:
                            list_riskDriversMP = updateRiskDriversMP(list_riskDriversMP, t, driver, driver1, risicoType)
                if (risicoType == '208'):
                    list_riskDriversMP = [1]
                    values201 = berekenDrivers(datum, modelpunt, methodeRiskDriver, renteCurve, risicoType, True)
                    values208 = berekenDrivers(datum, modelpunt, methodeRiskDriver, renteCurve, risicoType, False)
                    inflatie = berekenInflatie(modelpunt)
                    for t in range(1, 100):
                        driver = (1+inflatie)**t / (1+inflatie+schokInflatie)**t * values208[t] - values201[t]
                        if t == 1:
                            driver1 = driver
                        else:
                            list_riskDriversMP = updateRiskDriversMP(list_riskDriversMP, t, driver, driver1, risicoType)
                if (risicoType == '209'):
                    list_riskDriversMP = [1]
                    list_riskDriversMP += [0] * 98 # risk drivers worden later berekend obv SCR202 en SCR204
                if (risicoType == '210'):
                    list_riskDriversMP = [1]
                    indicatorCorrectie = df_methodenRiskDrivers.loc[df_methodenRiskDrivers['modelpunt'] == modelpunt, 'indicatorCorrectie'].values[0]
                    indicatorORUnitLinked = True
                    if (df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'nominaal'].values[0] == 1) | (df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'unitLinkedNaarNominaalOperationeelRisico'].values[0] == 1):
                        indicatorORUnitLinked = False
                    for t in range(1, 100):    
                        if indicatorORUnitLinked == True: 
                            driver = df_driverInput.loc[(df_driverInput['datum'] == datumKwartaal) & (df_driverInput['risicoType'] == '201') & (df_driverInput['modelpunt'] == modelpunt) & ((df_driverInput['variabele'] == 'FN_AANTALLYF') | (df_driverInput['variabele'] == 'FN_AANTALPB') | (df_driverInput['variabele'] == 'FN_AANTALPV')), 12*t].sum()
                        else:
                            value1 = df_driverInput.loc[(df_driverInput['datum'] == datumKwartaal) & (df_driverInput['modelpunt'] == modelpunt) & (df_driverInput['variabele'] == 'FN_PPREM_BRU'), 12*t].values[0]
                            value2 = berekenValueNaCorrectie(t, modelpunt, indicatorCorrectie)
                            driver = max(ORPremieRiskDriver * value1, ORLiabilityValueRiskDriver * value2)   
                        if t == 1:
                            driver1 = driver
                        else:
                            list_riskDriversMP = updateRiskDriversMP(list_riskDriversMP, t, driver, driver1, risicoType)
                df_riskDrivers[datum, risicoType, renteCurve].loc[df_riskDrivers[datum, risicoType, renteCurve]['modelpunt'] == modelpunt, 0:] = list_riskDriversMP  

In [62]:
def berekenProjectieSCRVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves(datum):
    for renteCurve in renteCurves:
        for risicoType in risicoTypes[1:10]:
            df_projectieSCR[datum, risicoType, renteCurve] = multiplyTwoDataFrames(df_riskDrivers[datum, risicoType, renteCurve], df_SCR[datum, risicoType, 'RM'][['modelpunt', renteCurve]])

    # de projectie van SCR209 kan nu worden afgeleid uit die van SCR202 en SCR204                                                                               
    for renteCurve in renteCurves:
        for modelpunt in modelpuntNamen:
            SCR2090 = df_projectieSCR[datum, '209', renteCurve].loc[df_projectieSCR[datum, '209', renteCurve]['modelpunt'] == modelpunt,0].values[0]
            SCR2020 = df_projectieSCR[datum, '202', renteCurve].loc[df_projectieSCR[datum, '202', renteCurve]['modelpunt'] == modelpunt,0].values[0]
            SCR2040 = df_projectieSCR[datum, '204', renteCurve].loc[df_projectieSCR[datum, '204', renteCurve]['modelpunt'] == modelpunt,0].values[0]                                                                   
            #print(modelpunt, SCR2020, SCR2040)
            listSCR209 = [SCR2090]
            if (SCR2020 + SCR2040) == 0:
                listSCR209 += [0] * 98 
            else:
                for t in range(1, 99):
                    SCR202t = df_projectieSCR[datum, '202', renteCurve].loc[df_projectieSCR[datum, '202', renteCurve]['modelpunt'] == modelpunt,t].values[0]
                    SCR204t = df_projectieSCR[datum, '204', renteCurve].loc[df_projectieSCR[datum, '204', renteCurve]['modelpunt'] == modelpunt,t].values[0] 
                    SCR209 = SCR2090 * (SCR202t + SCR204t) / (SCR2020 + SCR2040)
                    listSCR209 += [SCR209]
            df_projectieSCR[datum, '209', renteCurve].loc[df_projectieSCR[datum, '209', renteCurve]['modelpunt'] == modelpunt, 0:] = listSCR209

In [63]:
def aggregeerProjectieSCRVoorAlleRisicoTypesOverAlleModelpuntenVoorAlleRenteCurves(datum):
    for renteCurve in renteCurves:
        i = 0
        for risicoType in risicoTypes[1:]:
            df = df_projectieSCR[datum, risicoType, renteCurve].sum(axis=0, skipna = True, numeric_only = True)
            df_projectieSCRGeaggregeerd[datum, renteCurve].iloc[i,1:] = df
            i += 1

In [64]:
def bepaalSCRLapseName(df_SCRLapse, t):
    SCRLapseName = '205'
    if df_SCRLapse.loc[df_SCRLapse['risicoType'] == '206', t].values[0] > df_SCRLapse.loc[df_SCRLapse['risicoType'] == '205', t].values[0]:
        SCRLapseName = '206'
    if df_SCRLapse.loc[df_SCRLapse['risicoType'] == '207', t].values[0] > df_SCRLapse.loc[df_SCRLapse['risicoType'] == '206', t].values[0]:
        SCRLapseName = '207'   
    return SCRLapseName


In [65]:
def berekenSCRLeven(arr_SCRs):
    SCRLeven = np.matmul(np.matmul(arr_SCRs, correlatiesSCRLeven), arr_SCRs)**0.5
    return SCRLeven    

In [66]:
def aggregeerProjectieSCROverAlleRisicoTypesOverAlleModelpuntenVoorAlleRenteCurves(datum):
    for renteCurve in renteCurves:
        list_SCRLevenPlusOR = []
        for t in range(0, 99):
            df_SCRLapse = df_projectieSCRGeaggregeerd[datum, renteCurve].loc[(df_projectieSCRGeaggregeerd[datum, renteCurve]['risicoType'] == '205') | (df_projectieSCRGeaggregeerd[datum, renteCurve]['risicoType'] == '206') | (df_projectieSCRGeaggregeerd[datum, renteCurve]['risicoType'] == '207'), ['risicoType', t]]
            SCRLapseName = bepaalSCRLapseName(df_SCRLapse, t)
            list_SCRNames = ['202', '203', '204', SCRLapseName, '208', 'revisie', '209']
            list_SCRs = []
            for SCRName in list_SCRNames:
                if SCRName == 'revisie':
                    list_SCRs += [0]
                else:
                    list_SCRs += [df_projectieSCRGeaggregeerd[datum, renteCurve].loc[df_projectieSCRGeaggregeerd[datum, renteCurve]['risicoType'] == SCRName, t].values[0]]
            arr_SCRs = np.array(list_SCRs)
            SCRLeven = berekenSCRLeven(arr_SCRs)
            OR = df_projectieSCRGeaggregeerd[datum, renteCurve].loc[df_projectieSCRGeaggregeerd[datum, renteCurve]['risicoType'] == '210', t].values[0]
            list_SCRLevenPlusOR += [SCRLeven + OR]
        df_projectieSCRTotaal[datum].loc[df_projectieSCRTotaal[datum]['renteCurve'] == renteCurve, 0:] = list_SCRLevenPlusOR    

In [67]:
def berekenRisicoMargesVoorAlleRenteCurves(datum):
    for renteCurve in renteCurves:
        rentesRenteCurve = df_curves['ExclusiefVA'].loc[(df_curves['ExclusiefVA']['datum'] == datum) & (df_curves['ExclusiefVA']['Currency'] == 'EUR'), renteCurve].values
        risicoMarge = 0
        for t in range(0, 99):
            SCRTotaal = df_projectieSCRTotaal[datum].loc[df_projectieSCRTotaal[datum]['renteCurve'] == renteCurve, t].values[0]
            risicoMarge += SCRTotaal / (1 + rentesRenteCurve[t])**(t+1) # moet volgens mij t ipv t+1 zijn, maar zo wordt nu eenmaal risicomarge door SSC berekend
        df_risicoMarges[datum].loc[df_risicoMarges[datum]['renteCurve'] == renteCurve, 'risicoMarge'] = risicoMarge * CoC 

In [68]:
start_time = timeit.default_timer()
# initialiseer de tuple met indicator voor berekeningen tbv SCR of RM
SCRofRM = ('SCR', 'RM')

# initialiseer de tuple met rentecurves die moeten worden doorgerekend
renteCurves = list(df_curves['InclusiefVA'].columns)
renteCurves.remove(renteCurveBasis)
renteCurves = tuple(renteCurveBasis)
renteCurves = renteCurves[3:]
renteCurves = (renteCurveBasis,) + renteCurves

# vereenvoudigingen tijdens ontwikkelingsfase
renteCurves = ('SII_basis', 'swap.cra.zero.min100bp.va.sw360', 'swap.cra.zero.plus100bp.va.sw360',)
#renteCurves = ('SII_basis',)
#risicoTypes = ('201', '201UL205_208',)
#risicoTypes = ('208',)
#risicoTypes = ('201',)
#risicoTypes = ('201', '207',)
#modelpuntNamen = ('ANWH', 'EABD', 'EABV',) # nominaal
#modelpuntNamen = ('FRGR', 'FRKL', 'FRKR',) # unit linked
#modelpuntNamen = ('ANWH', 'EABD', 'EABV', 'FRGR', 'FRKL', 'FRKR',) # nominaal en unit linked
#modelpuntNamen = ('ISAK', 'ISAP', 'ISUD', 'ISZA', 'IXAD', 'IXUD',) # currency
#modelpuntNamen = ('EADV',)
#modelpuntNamen = ('EANW',)
#modelpuntNamen = ('IBSH',)
#modelpuntNamen = ('EACG',)
#modelpuntNamen = ('GAGS',)
#modelpuntNamen = ('FRGL',)
#modelpuntNamen = ('FRKL',)
#modelpuntNamen = ('ISSH',)
                  
# initialiseer lege dataframes met modelpunt namen die moeten worden gevuld met nog te berekenen waarden
df_LVHandCorrectie = pd.DataFrame()
df_LVHandCorrectie['modelpunt'] = modelpuntNamen
df_LV = dict(dict(dict()))
df_MWSPL = dict(dict())
df_SCR = dict(dict(dict()))
df_LACTPMP = dict(dict(dict()))
df_riskDrivers = dict(dict(dict()))
df_projectieSCR = dict(dict(dict()))
df_correctieLVFactoren = dict()
df_correctieMWSPLFactoren = dict()
df_SCRGeaggregeerd = dict(dict())
df_projectieSCRGeaggregeerd = dict(dict())
df_LVBasis = dict()
df_ULProfit = dict()
df_projectieSCRTotaal = dict()
df_risicoMarges = dict()
for datum in data:
    df_projectieSCRTotaal[datum] = pd.DataFrame()
    df_projectieSCRTotaal[datum]['renteCurve']  =renteCurves
    df_risicoMarges[datum] = pd.DataFrame()
    df_risicoMarges[datum]['renteCurve']  = renteCurves
    df_risicoMarges[datum]['renteCurve']['risicoMarge'] = 0
    for col in range(0,99):
        df_projectieSCRTotaal[datum][col] = 0    
    for risicoType in risicoTypes:
        df_MWSPL[datum, risicoType] = pd.DataFrame()
        df_MWSPL[datum, risicoType]['modelpunt'] = modelpuntNamen        
        for purpose in SCRofRM:
            df_LV[datum, risicoType, purpose] = pd.DataFrame()
            df_LV[datum, risicoType, purpose]['modelpunt'] = modelpuntNamen           
            df_SCR[datum, risicoType, purpose] = pd.DataFrame()
            df_SCR[datum, risicoType, purpose]['modelpunt'] = modelpuntNamen
            df_LACTPMP[datum, risicoType, purpose] = pd.DataFrame()
            df_LACTPMP[datum, risicoType, purpose]['modelpunt'] = modelpuntNamen
        for renteCurve in renteCurves:
            df_riskDrivers[datum, risicoType, renteCurve] = pd.DataFrame()
            df_riskDrivers[datum, risicoType, renteCurve]['modelpunt'] = modelpuntNamen           
            for col in range(0,99):
                df_riskDrivers[datum, risicoType, renteCurve][col] = 0
    for purpose in SCRofRM:
        df_SCRGeaggregeerd[datum, purpose] = pd.DataFrame(columns = renteCurves)
        df_SCRGeaggregeerd[datum, purpose].insert(0, 'risicoType', risicoTypes[1:])
    for renteCurve in renteCurves:
        df_projectieSCRGeaggregeerd[datum, renteCurve] = pd.DataFrame()
        df_projectieSCRGeaggregeerd[datum, renteCurve].insert(0, 'risicoType', risicoTypes[1:])
        for col in range(0,99):
            df_projectieSCRGeaggregeerd[datum, renteCurve][col] = 0     
for risicoType in risicoTypes:            
    df_correctieLVFactoren[risicoType] = pd.DataFrame()
    df_correctieLVFactoren[risicoType]['modelpunt'] = modelpuntNamen
    df_correctieMWSPLFactoren[risicoType] = pd.DataFrame()
    df_correctieMWSPLFactoren[risicoType]['modelpunt'] = modelpuntNamen


# BEREKENINGEN VOOR WAARDE VOORZIENING
# stap 1: bereken de LV (liability value) handmatige correcties voor alle modelpunten
berekenLiabilityValueHandCorrectie()
               
# stap 2a: bereken de LV per (laatste) kwartaal per risicotype voor alle modelpunten en rente curves inclusief VA (tbv bepaling van correctiefactoren)
berekenLiabilityValueVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves(datumKwartaal, 'SCR')

# stap 2b: bereken de correctiefactoren voor de LV per (laatste) kwartaal per risicotype voor alle modelpunten en rente curves inclusief VA
# stap 2b: opdat wordt aangesloten met de LV uit Gamma voor de renteCurveBasis
berekenCorrectieLiabilityValueFactorenVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves()

# stap 2c: bereken de LV per actuele datum per risicotype voor alle modelpunten en rente curves inclusief VA 
berekenLiabilityValueVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves(datumActueel, 'SCR')    
    
# stap 2d: bereken de totale LV per actuele datum voor 201 voor alle rente curves (inclusief VA)
berekenLiabilityValueVoorBasisRisicoOverAlleModelpuntenVoorAlleRenteCurves(datumActueel)

# stap 2e: bereken de unit linked profit per actuele datum voor 201 voor alle rente curves (inclusief VA)
berekenUnitLinkedProfitVoorBasisRisicoOverAlleModelpuntenVoorAlleRenteCurves(datumActueel)


# BEREKENINGEN VOOR SCR
# stap 3a: bereken de meerwaarde spaarlossen per (laatste) kwartaal per risicotype voor alle modelpunten en rente curves (tbv bepaling van correctiefactoren)
berekenMeerwaardeSpaarlossenVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves(datumKwartaal)

# stap 3b: bereken de correctiefactoren voor de meerwaarde spaarlossen per (laatste) kwartaal per risicotype voor alle modelpunten en rente curves
# stap 3b: opdat wordt aangesloten met de meerwaarde spaarlossen uit Gamma voor de renteCurveBasis
berekenCorrectieMeerwaardeSpaarlossenFactorenVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves()

# stap 3c: bereken de meerwaarde spaarlossen per actuele datum per risicotype voor alle modelpunten en rente curves
berekenMeerwaardeSpaarlossenVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves(datumActueel)

# stap 4: bereken de SCR per actuele datum per risicotype (exclusief operationeel risico) voor alle modelpunten en rente curves inclusief VA
berekenSCRAnteLACTPVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves(datumActueel, 'SCR')

# stap 5: bereken de LACTP per actuele datum per risicotype (exclusief operationeel risico) voor alle modelpunten en rentecurves inclusief VA
berekenLACTPVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves(datumActueel, 'SCR')

# stap 6: bereken de SCR na LACTP per actuele datum per risicotype (exclusief operationeel risico) voor alle modelpunten en rentecurves inclusief VA
berekenSCRVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves(datumActueel, 'SCR')

# stap 7: bereken het operationeel risico per actuele datum voor alle modelpunten en rente curves inclusief VA
berekenSCROperationeelVoorAlleModelpuntenVoorAlleRenteCurves(datumActueel, 'SCR')

# stap 8: aggregeer de SCR per actuele datum per risicotype over alle modelpunten voor iedere iedere curve inclusief VA
aggregeerSCRVoorAlleRisicoTypesOverAlleModelpuntenVoorAlleRenteCurves(datumActueel, 'SCR')


# BEREKENINGEN VOOR RISICOMARGE
# stap 2c: bereken de LV per actuele datum per risicotype voor alle modelpunten en rente curves inclusief VA 
berekenLiabilityValueVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves(datumActueel, 'RM')

# stap 4: bereken de SCR per actuele datum per risicotype (exclusief operationeel risico) voor alle modelpunten en rente curves inclusief VA
berekenSCRAnteLACTPVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves(datumActueel, 'RM')

# stap 5: bereken de LACTP per actuele datum per risicotype (exclusief operationeel risico) voor alle modelpunten en rentecurves inclusief VA
berekenLACTPVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves(datumActueel, 'RM')

# stap 6: bereken de SCR na LACTP per actuele datum per risicotype (exclusief operationeel risico) voor alle modelpunten en rentecurves inclusief VA
berekenSCRVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves(datumActueel, 'RM')

# stap 7: bereken het operationeel risico per actuele datum voor alle modelpunten en rente curves inclusief VA
berekenSCROperationeelVoorAlleModelpuntenVoorAlleRenteCurves(datumActueel, 'RM')

# stap 9: bereken de risicodrivers per actuele datum per risicotype voor alle modelpunten voor iedere curve exclusief VA
berekenRiskDriversVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves(datumActueel)

# stap 10:
berekenProjectieSCRVoorAlleRisicoTypesVoorAlleModelpuntenVoorAlleRenteCurves(datumActueel)

# stap 11
aggregeerProjectieSCRVoorAlleRisicoTypesOverAlleModelpuntenVoorAlleRenteCurves(datumActueel)

# stap 12
aggregeerProjectieSCROverAlleRisicoTypesOverAlleModelpuntenVoorAlleRenteCurves(datumActueel)

# stap 13
berekenRisicoMargesVoorAlleRenteCurves(datumActueel)

elapsed = timeit.default_timer() - start_time
elapsed

1109.397240600083

## Output

### Waarde Leven voorzieningen voor alle rentecurves

In [69]:
df_LVBasis[20210630]

,SII_basis,swap.cra.zero.min100bp.va.sw360,swap.cra.zero.plus100bp.va.sw360
ULCollectief,"5,428,783,275.0303","5,522,548,712.2956","5,334,883,597.4039"
ULCollectiefGarantie,"464,229,242.4900","429,681,670.9687","490,704,849.0533"
ULIndividueel,"4,099,444,451.7361","4,168,776,065.3654","4,030,024,979.6559"
ULIndividueelGarantie,"1,052,368,675.2968","1,061,286,744.3035","1,041,754,698.5187"
nominaalCollectief,"20,847,605,259.2042","24,300,317,178.0794","18,004,171,114.0806"
nominaalIndividueel,"15,567,574,182.7585","17,639,472,185.8880","13,845,611,484.6782"
total,"47,460,005,086.5158","53,122,082,556.9006","42,747,150,723.3907"


### Waarde Unit linked profit voor alle rentecurves

In [70]:
df_ULProfit[20210630]

,SII_basis,swap.cra.zero.min100bp.va.sw360,swap.cra.zero.plus100bp.va.sw360
ULCollectief,"-16,235,728.0998","-110,001,165.3651","77,663,949.5266"
ULCollectiefGarantie,"-66,318,006.9000","-31,770,435.3787","-92,793,613.4633"
ULIndividueel,"191,649,438.6894","122,317,825.0602","261,068,910.7696"
ULIndividueelGarantie,"24,564,741.3230","15,646,672.3163","35,178,718.1011"
total,"133,660,445.0126","-3,807,103.3673","281,117,964.9340"


### SCR kapitalen (incl VA) voor alle rentecurves

In [71]:
df_SCRGeaggregeerd[20210630, 'SCR']

,risicoType,SII_basis,swap.cra.zero.min100bp.va.sw360,swap.cra.zero.plus100bp.va.sw360
0,202,"255,948,358.8577","264,485,936.5222","247,048,734.9780"
1,203,"1,215,427,075.2920","1,489,013,842.2934","994,813,887.1182"
2,204,"8,460,504.7629","8,670,100.1833","8,259,772.2045"
3,205,"105,810,185.4724","139,006,084.4652","80,345,735.3557"
4,206,"88,743,373.8577","86,839,040.8264","90,639,681.4283"
5,207,"291,890,281.2068","241,905,129.4683","372,455,153.8679"
6,208,"649,463,407.2681","772,818,666.5350","549,720,272.1120"
7,209,"111,079,217.2947","115,347,652.9203","107,647,137.8631"
8,210,"181,766,180.5661","206,511,592.4539","161,293,272.1058"


### SCR kapitalen (excl VA) voor alle rentecurves

#### SII_basis

In [72]:
df_projectieSCRGeaggregeerd[20210630, 'SII_basis']

,risicoType,0,1,2,3,4,5,6,7,8,...,89,90,91,92,93,94,95,96,97,98
0,202,"257,514,983.9155","249,687,898.7432","242,370,028.1019","234,306,321.4828","226,303,817.3823","218,441,848.4914","210,486,600.7953","201,985,451.6672","192,953,265.3924",...,"170,935.1638","114,735.2029","75,784.7023","49,355.0822","32,225.6798","20,420.0852","13,109.9428","8,356.8494","4,976.8019","2,306.8504"
1,203,"1,227,966,037.8696","1,205,275,362.3291","1,179,455,472.7529","1,153,842,755.3790","1,128,365,615.6377","1,102,919,603.2385","1,076,181,283.9599","1,048,338,603.1683","1,019,394,183.7982",...,83.8281,65.6090,50.7414,38.5191,28.4933,20.3864,14.0106,9.2275,5.8026,3.4794
2,204,"8,466,834.2627","2,402,038.1960","1,253,374.6655","943,004.8330","801,917.0423","696,587.7966","583,883.1031","481,255.8238","398,106.0963",...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,205,"110,237,872.9545","93,266,429.4512","79,098,459.4537","67,084,138.1286","56,615,762.3126","47,603,734.5494","39,898,122.8194","33,342,233.3279","27,758,265.1978",...,23.0532,15.1078,9.6729,6.0392,3.6619,2.1374,1.1781,0.5840,0.2202,0.0000
4,206,"86,695,513.9853","77,251,116.7331","68,775,717.0868","61,113,135.9645","54,186,188.1476","47,897,294.5328","42,162,645.0117","36,978,926.9704","32,338,380.8885",...,11.7505,6.5891,3.5572,1.8482,0.9231,0.4414,0.1993,0.0814,0.0257,0.0000
5,207,"278,994,836.8727","242,063,478.7638","216,342,609.9147","192,924,419.0976","171,755,044.2361","152,606,970.8758","135,186,492.1200","119,451,234.4644","105,415,937.2999",...,6.6776,3.3501,1.5966,0.7196,0.2943,0.1149,0.0390,0.0106,0.0017,0.0000
6,208,"656,924,089.5523","616,107,077.9761","579,838,455.6765","546,212,119.8651","515,021,988.7647","486,190,460.7974","459,097,852.1532","433,796,921.2666","410,492,905.1912",...,"64,451.5927","48,317.5507","35,483.9434","25,374.7075","17,523.1607","11,537.2457","7,072.1796","3,824.4163","1,536.9542",0.0000
7,209,"111,303,605.8490","65,790,689.2958","62,894,891.7182","59,832,363.7731","56,750,954.8303","53,778,121.9703","50,702,556.5257","47,351,325.4261","43,571,091.1003",...,"9,727.5464","6,375.0517","4,098.6725","2,593.9032","1,644.0655","1,016.0255",652.7344,424.4334,253.9410,107.3135
8,210,"182,955,134.0840","175,200,453.6008","169,483,751.8258","163,491,665.4516","158,053,151.2471","152,131,656.6891","146,550,937.8743","141,049,200.9971","135,515,330.2781",...,"87,976.7863","65,152.2607","47,550.4947","34,482.4193","24,540.9141","17,077.5441","11,674.8299","7,822.1594","5,068.6706","3,192.9222"


#### swap.cra.zero.min100bp.va.sw360

In [73]:
df_projectieSCRGeaggregeerd[20210630, 'swap.cra.zero.min100bp.va.sw360']

,risicoType,0,1,2,3,4,5,6,7,8,...,89,90,91,92,93,94,95,96,97,98
0,202,"264,647,169.4917","256,631,003.5830","249,072,743.4359","240,725,292.4409","232,436,668.9953","224,292,731.9285","216,022,008.6685","207,177,827.7635","197,736,623.5093",...,"178,123.9976","118,901.5455","78,045.3196","50,487.6809","32,679.6829","20,552.3049","13,224.0305","8,482.0635","5,021.5017","2,258.3943"
1,203,"1,504,539,831.0733","1,477,329,733.0938","1,446,204,379.8932","1,415,348,756.0925","1,384,583,803.7590","1,353,838,694.7936","1,321,460,160.5371","1,287,699,828.0568","1,252,576,430.7253",...,104.1959,81.4230,62.7941,47.7390,35.0833,25.1006,17.2502,11.3609,7.1442,4.2838
2,204,"8,677,057.3190","2,474,107.7991","1,303,188.0962","984,368.2539","838,330.8323","728,613.3819","611,484.3499","504,770.3842","418,085.5580",...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,205,"144,259,588.5645","121,920,224.0285","103,282,022.4237","87,569,785.9679","73,956,360.8606","62,304,599.5940","52,379,276.1584","43,960,331.8144","36,822,365.5918",...,39.9733,25.6628,16.1208,9.8929,5.9082,3.4037,1.8553,0.9113,0.3411,0.0000
4,206,"83,244,411.3553","73,609,597.9026","65,006,774.3270","57,290,036.3469","50,368,561.0851","44,130,283.9548","38,488,582.8386","33,431,536.9857","28,940,331.5281",...,8.9434,5.0573,2.7504,1.4379,0.7217,0.3463,0.1566,0.0639,0.0201,0.0000
5,207,"227,268,615.1338","199,954,107.6339","175,885,383.4076","154,306,795.2280","135,032,360.4729","117,749,331.4388","102,182,649.1565","88,252,268.8994","75,918,101.4055",...,4.7148,2.3733,1.1320,0.5091,0.2127,0.0819,0.0278,0.0076,0.0012,0.0000
6,208,"780,287,052.9424","728,095,445.8721","681,134,516.0664","637,735,034.5474","597,632,512.7231","560,689,758.1704","526,139,693.5255","494,006,396.2245","464,484,182.1709",...,"60,946.7377","45,532.4731","33,328.9172","23,759.8735","16,360.7386","10,743.7608","6,570.6284","3,546.0483","1,422.4511",0.0000
7,209,"115,843,045.8906","65,117,854.5193","62,202,790.4912","59,140,243.3178","56,057,735.4956","53,081,258.2963","49,994,807.3831","46,634,615.6194","42,843,677.3322",...,"11,411.4009","7,891.2812","5,374.3606","3,599.4050","2,386.0204","1,528.9955",986.4339,623.6328,363.9397,169.0437
8,210,"207,945,962.9541","199,818,034.2105","193,735,580.9241","187,333,743.1730","181,502,536.4200","175,124,339.6659","169,080,785.5380","163,107,682.8598","157,086,228.6593",...,"131,032.5523","98,464.0393","72,971.4136","53,654.5859","38,757.3757","27,414.9747","19,057.7740","12,995.1195","8,595.1672","5,529.6450"


#### swap.cra.zero.plus100bp.va.sw360

In [74]:
df_projectieSCRGeaggregeerd[20210630, 'swap.cra.zero.plus100bp.va.sw360']

,risicoType,0,1,2,3,4,5,6,7,8,...,89,90,91,92,93,94,95,96,97,98
0,202,"246,967,587.5976","239,403,589.4846","232,398,244.4977","224,694,807.8896","217,059,521.8957","209,557,445.6525","201,988,001.3437","193,898,193.5328","185,330,121.3163",...,"151,522.8598","100,806.4617","65,958.7533","42,572.2165","27,686.7803","17,496.4564","11,198.0787","7,150.9650","4,309.7577","2,004.2727"
1,203,"1,005,014,843.0771","985,952,704.2103","964,422,741.7909","943,058,901.7045","921,863,364.4528","900,706,328.4533","878,532,725.8383","855,477,655.6979","831,535,122.0849",...,68.1681,53.3664,41.2794,31.3390,23.1833,16.5877,11.4002,7.5084,4.7216,2.8312
2,204,"8,265,685.2771","2,335,704.3848","1,208,279.7629","905,748.5509","769,136.8758","667,758.9831","559,066.8390","460,144.2838","380,182.7070",...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,205,"83,953,152.2660","71,201,622.5023","60,549,691.8985","51,431,873.4891","43,429,465.5175","36,482,867.2212","30,509,031.5380","25,405,131.9521","21,032,262.3013",...,10.8176,7.2491,4.7358,3.0100,1.8536,1.0964,0.6111,0.3058,0.1162,0.0000
4,206,"87,359,841.6917","78,214,565.7607","69,985,553.3164","62,502,875.6467","55,698,909.0805","49,486,332.1726","43,785,572.2863","38,599,571.2889","33,928,774.2406",...,17.2857,9.6438,5.1860,2.6875,1.3407,0.6410,0.2895,0.1184,0.0374,0.0000
5,207,"358,386,065.8880","305,766,742.0793","279,270,228.8263","254,944,485.3134","232,796,874.2120","212,699,635.3211","194,334,733.9427","177,701,751.7135","162,918,554.5288",...,-9.1222,-5.6190,-3.3165,-1.8819,-1.0436,-0.5403,-0.2661,-0.1197,-0.0437,0.0000
6,208,"554,516,450.7576","522,176,763.1340","494,026,570.3706","467,882,794.9863","443,572,609.5925","421,053,575.1086","399,819,052.4918","379,934,602.2069","361,602,705.8627",...,"60,002.6581","44,829.1490","32,815.8889","23,395.4157","16,110.4856","10,579.7505","6,470.4322","3,491.9785","1,400.7381",0.0000
7,209,"106,188,353.2507","67,433,148.1797","64,508,770.2219","61,401,319.5332","58,276,582.8727","55,260,676.2387","52,149,283.0863","48,765,487.2481","44,958,431.7738",...,"13,479.7395","9,221.8483","6,214.6664","4,125.9238","2,728.5893","1,746.2546","1,123.0891",710.6089,420.0569,197.5053
8,210,"162,296,936.9916","154,868,341.5110","149,471,241.2074","143,836,572.0708","138,738,497.0136","133,211,852.9467","128,028,604.3902","122,935,473.9093","117,835,730.8536",...,"72,086.8705","54,482.1739","40,738.1696","30,362.3150","22,260.4942","15,991.0957","11,286.1524","7,794.4156","5,207.4892","3,377.3353"


### Risicomarge voor alle rentecurves

In [75]:
pd.options.display.float_format = '{:,.0f}'.format
df_risicoMarges[20210630]

,renteCurve,risicoMarge
0,SII_basis,"1,996,539,476"
1,swap.cra.zero.min100bp.va.sw360,"2,669,052,720"
2,swap.cra.zero.plus100bp.va.sw360,"1,554,505,922"


## Verificatie

In [76]:
def maakScatter(list_Gamma, list_Python, list_Description, titel):
    sourceSpread = ColumnDataSource(
            data=dict(
                xLV=list_Gamma,
                yLV=list_Python,
                description=list_Description,))
    hover = HoverTool(
            tooltips=[
                ("(x,y)", "@xLV{(0.000 a)}, @yLV{(0.000 a)})"),
                ("desc", "@description"),])
    p = figure(plot_width=800, plot_height=500, tools = [hover, 'box_zoom', 'pan', 'wheel_zoom', 'reset'], title=titel)
    p.circle('xLV', 'yLV', size=5, source=sourceSpread)
    lineStart = min(list_Gamma)
    lineEnd = max(list_Gamma)
    x = [lineStart,lineEnd]
    y = [lineStart,lineEnd]
    p.line(x, y,  line_width=1)
    p.xaxis.axis_label = "Gamma"
    p.yaxis.axis_label = "Python"
    p.xaxis.formatter = NumeralTickFormatter(format="0 a")
    p.yaxis.formatter = NumeralTickFormatter(format="0 a")
    show(p)    

In [77]:
def maakLine(risicoType):
    x = list(range(0,99))
    y1 = df_SCRProjecties.loc[(df_SCRProjecties['renteCurve'] == 'SII_basis') & (df_SCRProjecties['risicoType'] == risicoType), 0:].values[0].tolist()
    y2 = df_projectieSCRGeaggregeerd[20210630, 'SII_basis'].loc[df_projectieSCRGeaggregeerd[20210630, 'SII_basis']['risicoType'] == str(risicoType), 0:].values[0].tolist()
    y3 = df_SCRProjecties.loc[(df_SCRProjecties['renteCurve'] == 'swap.cra.zero.min100bp.va.sw360') & (df_SCRProjecties['risicoType'] == risicoType), 0:].values[0].tolist()
    y4 = df_projectieSCRGeaggregeerd[20210630, 'swap.cra.zero.min100bp.va.sw360'].loc[df_projectieSCRGeaggregeerd[20210630, 'swap.cra.zero.min100bp.va.sw360']['risicoType'] == str(risicoType), 0:].values[0].tolist()
    y5 = df_SCRProjecties.loc[(df_SCRProjecties['renteCurve'] == 'swap.cra.zero.plus100bp.va.sw360') & (df_SCRProjecties['risicoType'] == risicoType), 0:].values[0].tolist()
    y6 = df_projectieSCRGeaggregeerd[20210630, 'swap.cra.zero.plus100bp.va.sw360'].loc[df_projectieSCRGeaggregeerd[20210630, 'swap.cra.zero.plus100bp.va.sw360']['risicoType'] == str(risicoType), 0:].values[0].tolist()

    p = figure(title="SCR projectie risico type", x_axis_label="jaar", y_axis_label="SCR", width=800, height=500)
    p.line(x, y1, legend_label="Gamma basis", color = "blue", line_width=2, line_dash = "solid")
    p.line(x, y2, legend_label="Python basis", color = "blue", line_width=2, line_dash = "dashed")
    p.line(x, y3, legend_label="Gamma swap.cra.zero.min100bp.va.sw360", color = "red", line_width=2, line_dash = "solid")
    p.line(x, y4, legend_label="Python swap.cra.zero.min100bp.va.sw360", color = "red", line_width=2, line_dash = "dashed")
    p.line(x, y5, legend_label="Gamma swap.cra.zero.plus100bp.va.sw360", color = "green", line_width=2, line_dash = "solid")
    p.line(x, y6, legend_label="Python swap.cra.zero.plus100bp.va.sw360", color = "green", line_width=2, line_dash = "dashed")
    p.xaxis.formatter = NumeralTickFormatter(format="0 a")
    p.yaxis.formatter = NumeralTickFormatter(format="0 a")
    show(p)    

### LV (excl ijking) per modelpunt per risico bij basis curve op kwartaal datum

In [78]:
list_LVGamma = []
list_LVPython = []
list_Description = []
for risicoType in risicoTypes:
    if (risicoType != '210'):
        for modelpunt in modelpuntNamen:            
            if risicoType != '207':
                LVGamma = df_waardenVoorzieningenGamma.loc[(df_waardenVoorzieningenGamma['datum'] == datumKwartaal) & (df_waardenVoorzieningenGamma['modelpunt'] == modelpunt), risicoType].values[0]
            if risicoType == '207':
                if df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'nominaal'].values[0] == 1:
                    if df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'nominaalUitvaart'].values[0] == 1:
                        AV = df_vloerenGamma.loc[df_vloerenGamma['modelpunt'] == modelpunt, 'afkoopVloer'].values[0]    
                        if df_vloerenGamma.loc[df_vloerenGamma['modelpunt'] == modelpunt, 'afkoopMethode'].values[0] == 2:
                            pvi = 1
                        else:
                            pvi = 0
                        PVGamma = df_vloerenGamma.loc[df_vloerenGamma['modelpunt'] == modelpunt, 'premieVrijmaakVloer'].values[0]
                        LVGamma = AV + pvi * PVGamma
                    elif df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'nominaalRest'].values[0] == 1:
                        AV = df_vloerenGamma.loc[df_vloerenGamma['modelpunt'] == modelpunt, 'afkoopVloer'].values[0]
                        meerwaardeSpaarlosGamma = df_meerwaardeSpaarlossenGamma.loc[(df_meerwaardeSpaarlossenGamma['datum'] == datumKwartaal) & (df_meerwaardeSpaarlossenGamma['modelpunt'] == modelpunt), risicoType].values[0]
                        LVGamma = AV + meerwaardeSpaarlosGamma
                    elif df_productSoorten.loc[df_productSoorten['modelpunt'] == modelpunt, 'nominaalCollectief'].values[0] == 1:
                        BVGamma = df_projectieBoekwaardeVoorziening.loc[(df_projectieBoekwaardeVoorziening['datum'] == datumKwartaal) & (df_projectieBoekwaardeVoorziening['risicoType'] == risicoType) & (df_projectieBoekwaardeVoorziening['modelpunt'] == modelpunt), 0].values[0] 
                        LVHandCorrectieGamma = df_waardenVoorzieningenGamma.loc[(df_waardenVoorzieningenGamma['datum'] == datumKwartaal) & (df_waardenVoorzieningenGamma['modelpunt'] == modelpunt), '207Hand'].values[0]   
                        LVGamma = BVGamma + LVHandCorrectieGamma
                    else:
                        LVGamma = 0
                else: # UNIT LINKED
                    AVGamma = df_vloerenGamma.loc[df_vloerenGamma['modelpunt'] == modelpunt, 'afkoopVloer'].values[0]
                    LVGamma = AVGamma                  
            LVPython = df_LV[datumKwartaal, risicoType, 'SCR'].loc[df_LV[datumKwartaal, risicoType, 'SCR']['modelpunt'] == modelpunt, renteCurveBasis].values[0]
            list_LVGamma += [LVGamma]
            list_LVPython += [LVPython]
            list_Description += [risicoType+'_'+modelpunt]    
            
maakScatter(list_LVGamma, list_LVPython, list_Description, "LV Python (excl. correctie op LV) versus LV Gamma (igv de renteCurveBasis)")

### MWSPL (excl ijking) per modelpunt per risico bij basis curve op kwartaal datum

In [79]:
list_MWSPLGamma = []
list_MWSPLPython = []
list_Description = []
for risicoType in risicoTypes:
    if (risicoType != '207') & (risicoType != '210'):
        for modelpunt in modelpuntNamen:
            MWSPLGamma = df_meerwaardeSpaarlossenGamma.loc[(df_meerwaardeSpaarlossenGamma['datum'] == datumKwartaal) & (df_meerwaardeSpaarlossenGamma['modelpunt'] == modelpunt), risicoType].values[0]
            MWSPLPython = df_MWSPL[datumKwartaal, risicoType].loc[df_MWSPL[datumKwartaal, risicoType]['modelpunt'] == modelpunt, renteCurveBasis].values[0]
            list_MWSPLGamma += [MWSPLGamma]
            list_MWSPLPython += [MWSPLPython]
            list_Description += [risicoType+'_'+modelpunt]

maakScatter(list_MWSPLGamma, list_MWSPLPython, list_Description, "MWSPL Python (excl. correctie op MWSPL) versus MWSPL Gamma (igv de renteCurveBasis)")

### SCR per modelpunt per risico bij curve basis op kwartaal datum

In [80]:
df_SCRS2basis = pd.read_excel(r"databestanden SSC/Verificatie data/SCR_S2basis.xlsx")
#df_SCRS2basis

In [81]:
list_SCRGamma = []
list_SCRPython = []
list_Description = []
for risicoType in risicoTypes:
    if (risicoType != '201'):   
        for modelpunt in modelpuntNamen:
            SCRGamma = df_SCRS2basis.loc[(df_SCRS2basis['modelpunt'] == modelpunt), int(risicoType)].values[0]
            SCRPython = df_SCR[datumKwartaal, risicoType, 'SCR'].loc[df_SCR[datumKwartaal, risicoType, 'SCR']['modelpunt'] == modelpunt, renteCurveBasis].values[0]
            list_SCRGamma += [SCRGamma]
            list_SCRPython += [SCRPython]
            list_Description += [risicoType+'_'+modelpunt]

maakScatter(list_SCRGamma, list_SCRPython, list_Description, "SCR Python versus SCR Gamma (igv renteCurveBasis)")

### SCR per modelpunt per risico bij curve min100bps op kwartaal datum

In [82]:
df_SCRMin100bps = pd.read_excel(r"databestanden SSC/Verificatie data/SCR_min100bps.xlsx")
#df_SCRMin100bps

In [83]:
list_SCRGamma = []
list_SCRPython = []
list_Description = []
for risicoType in risicoTypes:
    if (risicoType != '201'):
        for modelpunt in modelpuntNamen:
            SCRGamma = df_SCRMin100bps.loc[(df_SCRMin100bps['modelpunt'] == modelpunt), int(risicoType)].values[0]
            SCRPython = df_SCR[datumKwartaal, risicoType, 'SCR'].loc[df_SCR[datumKwartaal, risicoType, 'SCR']['modelpunt'] == modelpunt, 'swap.cra.zero.min100bp.va.sw360'].values[0]
            list_SCRGamma += [SCRGamma]
            list_SCRPython += [SCRPython]
            list_Description += [risicoType+'_'+modelpunt]

maakScatter(list_SCRGamma, list_SCRPython, list_Description, "SCR Python versus SCR Gamma (igv min100bps)")

### SCR per modelpunt per risico bij curve plus100bps op kwartaal datum

In [84]:
df_SCRPlus100bps = pd.read_excel(r"databestanden SSC/Verificatie data/SCR_plus100bps.xlsx")
#df_SCRPlus100bps

In [85]:
list_SCRGamma = []
list_SCRPython = []
list_Description = []
for risicoType in risicoTypes:
    if (risicoType != '201'):
        for modelpunt in modelpuntNamen:
            SCRGamma = df_SCRPlus100bps.loc[(df_SCRPlus100bps['modelpunt'] == modelpunt), int(risicoType)].values[0]
            SCRPython = df_SCR[datumKwartaal, risicoType, 'SCR'].loc[df_SCR[datumKwartaal, risicoType, 'SCR']['modelpunt'] == modelpunt, 'swap.cra.zero.plus100bp.va.sw360'].values[0]
            list_SCRGamma += [SCRGamma]
            list_SCRPython += [SCRPython]
            list_Description += [risicoType+'_'+modelpunt]

maakScatter(list_SCRGamma, list_SCRPython, list_Description, "SCR Python versus SCR Gamma (igv plus100bps)")

### SCR projectie risico type 202 bij curves basis, min100bps en plus100bps op kwartaal datum

In [86]:
df_SCRProjecties = pd.read_excel(r"databestanden SSC/Verificatie data/SCRProjecties.xlsx")
df_SCRProjecties

,renteCurve,risicoType,0,1,2,3,4,5,6,7,...,89,90,91,92,93,94,95,96,97,98
0,SII_basis,202,"256,314,831","248,567,956","241,248,030","233,225,903","225,266,641","217,448,378","209,536,072","201,075,295",...,"164,833","109,839","71,979","46,507","30,173","19,021","12,206","7,812","4,665","2,135"
1,swap.cra.zero.min100bp.va.sw360,202,"263,727,655","255,777,094","248,218,771","239,912,681","231,663,451","223,557,543","215,320,818","206,502,639",...,"177,086","118,372","77,813","50,407","32,663","20,560","13,233","8,485","5,021","2,261"
2,swap.cra.zero.plus100bp.va.sw360,202,"248,383,335","240,854,360","233,776,407","226,036,274","218,360,355","210,824,965","203,227,998","195,109,245",...,"154,012","102,388","66,941","43,172","28,051","17,713","11,337","7,242","4,359","2,024"
3,SII_basis,203,"1,227,863,686","1,205,177,464","1,179,361,629","1,153,752,928","1,128,279,614","1,102,837,457","1,076,102,850","1,048,263,823",...,84,66,51,39,28,20,14,9,6,3
4,swap.cra.zero.min100bp.va.sw360,203,"1,504,458,080","1,477,113,979","1,445,852,496","1,414,858,094","1,383,971,194","1,353,122,411","1,320,652,136","1,286,814,329",...,104,81,63,48,35,25,17,11,7,4
5,swap.cra.zero.plus100bp.va.sw360,203,"1,004,649,546","985,778,105","964,420,908","943,220,436","922,175,802","901,154,888","879,101,624","856,149,903",...,68,53,41,31,23,17,11,8,5,3
6,SII_basis,204,"8,466,659","2,401,787","1,253,069","942,738","801,692","696,397","583,721","481,118",...,0,0,0,0,0,0,0,0,0,0
7,swap.cra.zero.min100bp.va.sw360,204,"8,557,646","2,476,324","1,304,963","985,749","839,428","729,514","612,224","505,401",...,0,0,0,0,0,0,0,0,0,0
8,swap.cra.zero.plus100bp.va.sw360,204,"8,317,864","2,332,962","1,205,810","903,725","767,511","666,409","557,940","459,185",...,0,0,0,0,0,0,0,0,0,0
9,SII_basis,205,"109,634,655","92,861,586","78,815,159","66,877,071","56,457,781","47,479,023","39,797,256","33,259,872",...,23,15,10,6,4,2,1,1,0,0


In [87]:
maakLine(202)

### SCR projectie risico type 203 bij curves basis, min100bps en plus100bps op kwartaal datum

In [88]:
maakLine(203)

### SCR projectie risico type 204 bij curves basis, min100bps en plus100bps op kwartaal datum

In [89]:
maakLine(204)

### SCR projectie risico type 205 bij curves basis, min100bps en plus100bps op kwartaal datum

In [90]:
maakLine(205)

### SCR projectie risico type 206 bij curves basis, min100bps en plus100bps op kwartaal datum

In [91]:
maakLine(206)

### SCR projectie risico type 207 bij curves basis, min100bps en plus100bps op kwartaal datum

In [92]:
maakLine(207)

### SCR projectie risico type 208 bij curves basis, min100bps en plus100bps op kwartaal datum

In [93]:
maakLine(208)

### SCR projectie risico type 209 bij curves basis, min100bps en plus100bps op kwartaal datum

In [94]:
maakLine(209)

### SCR projectie risico type 210 bij curves basis, min100bps en plus100bps op kwartaal datum

In [95]:
maakLine(210)

### Risicomarge bij verschillende rente curves

In [96]:
df_risicoMargesGamma = pd.read_excel(r"databestanden SSC/Verificatie data/risicoMarges.xlsx")
df_risicoMargesGamma

,renteCurve,risicoMarge
0,SII_basis,"2,000,126,755"
1,swap.cra.zero.min100bp.va.sw360,"2,678,017,157"
2,swap.cra.zero.min50bp.va.sw360,"2,309,944,022"
3,swap.cra.zero.plus50bp.va.sw360,"1,748,614,787"
4,swap.cra.zero.plus100bp.va.sw360,"1,556,860,366"
5,swap.cra.zero.min150bp.va.sw360,"3,124,956,288"
6,swap.cra.zero.min200bp.va.sw360,"3,669,040,808"
7,swap.cra.zero.min250bp.va.sw360,"4,333,088,417"
8,swap.cra.zero.plus150bp.va.sw360,"1,409,664,513"
9,swap.cra.zero.plus200bp.va.sw360,"1,295,227,217"


In [97]:
list_RMGamma = []
list_RMPython = []
list_Description = []
for renteCurve in renteCurves:
    RMGamma = df_risicoMargesGamma.loc[(df_risicoMargesGamma['renteCurve'] == renteCurve), 'risicoMarge'].values[0]
    RMPython = df_risicoMarges[20210630].loc[(df_risicoMarges[20210630]['renteCurve'] == renteCurve), 'risicoMarge'].values[0]
    list_RMGamma += [RMGamma]
    list_RMPython += [RMPython]
    list_Description += [renteCurve]
maakScatter(list_RMGamma, list_RMPython, list_Description, "RM Python versus RM Gamma")